
# Parse

In [1]:
import sys
sys.path.append('..')

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from clib import data, tokenize, context_words, snippets

In [6]:
from pathlib import Path

DATA_PATH = Path('/kaggle/input/CORD-19-research-challenge/2020-03-13/')
DATA_PATH = Path('2020-03-13/').expanduser()

In [4]:
1/0 # do not overwrite

tokenize.cache_tokenized_papers(DATA_PATH, 'papers_v2.lj', only_some=True)

# Collocations

In [7]:
papers = tokenize.load_tokenized_papers('papers.jl')

In [8]:
full_stats = context_words.get_context_stats(papers, 10)

In [9]:
context_words.save_stats(full_stats, 'full_stats')

saving unigrams...



saving bigrams...


In [14]:
from tqdm.auto import tqdm

covid_papers = []
non_covid_papers = []

keywords = [
    ['covid'], ['coronavirus'], ['sars'], ['corona', 'virus'], 
    'severe acute respiratory syndrome'.split(), 'mers'.split(), 
    'middle east respiratory syndrome'.split()
]

for p in tqdm(papers):
    title = p['metadata.title'][0]['tokens']
    
    for kwds in keywords:
        if all([any([k in tok for tok in title]) for k in kwds]):
            covid_papers.append(p)
            break
    else:
        non_covid_papers.append(p)

len(covid_papers), len(non_covid_papers)

(1278, 11924)

In [11]:
non_covid_stats = context_words.get_context_stats(non_covid_papers, 10)

KeyboardInterrupt: 

In [ ]:
covid_stats = context_words.get_context_stats(covid_papers, 10)

In [ ]:
context_words.save_stats(non_covid_stats, 'non_covid_stats')
context_words.save_stats(covid_stats, 'covid_stats')

# Compute scores

In [32]:
full_stats = context_words.load_stats('full_stats')
covid_stats = context_words.load_stats('covid_stats')
non_covid_stats = context_words.load_stats('non_covid_stats')

FileNotFoundError: [Errno 2] No such file or directory: 'covid_stats/unigrams.jl'

In [16]:
len(full_stats.bigrams)

10858218

In [17]:
vocab = dict(sorted(full_stats.unigrams.items(), key=lambda x: -x[1])[:60000])

In [26]:
small_full_stats = context_words.ContextStats(
    unigrams=vocab, 
    bigrams={
        center: {
            word: score 
            for word, score in related.items()
            if word in vocab
        }
        for center, related in tqdm(full_stats.bigrams.items())
        if center in vocab
    }
)

In [27]:
scores = context_words.get_scores(small_full_stats)

In [14]:
comparative_scores, out_ref = context_words.get_comparative_scores(covid_stats, non_covid_stats)

In [29]:
def top(d, t, k=10):
    return sorted(d[t].items(), key=lambda x: -x[1])[:k]

In [54]:
keywords = [
    'isolation', 'quarantine', 'npi', 'funding', 'growth_rate', 'r0', 
    'health_care', 'capacity', 'mitigation', 'social_distancing', #'migration'
]

In [55]:
for k in keywords[-2:]:
    snippets.dm(f'**{k}**\n\n')
    for r, s in top(scores, k, k=10):
        snippets.dm(f'- {r}')

**mitigation**



- mitigation

- containment

- emissions

- strategies

...

- rigorous

- epicenter

- albany

- behaviours

- waning

In [35]:
from collections import defaultdict

In [33]:
l1 = top(scores, 'npi', 30)
l1

[('npi', 601.3635569022784),
 ('npis', 61.30913025184829),
 ('quadratic', 8.93688326196239),
 ('facemasks', 6.637902422834385),
 ('implementation', 5.360520795212643),
 ('cost', 3.744428171377962),
 ('strength', 2.637784261220575),
 ('lost', 1.9641980268738002),
 ('policy', 1.4548827228130157),
 ('exponential', 1.2610552510340653),
 ('cumulative', 1.0389760314001646),
 ('optimal', 1.0329229855491204),
 ('triggers', 0.8992446345070141),
 ('minimize', 0.782040865979919),
 ('death', 0.7567089124258483),
 ('terminal', 0.6792959097895032),
 ('effectiveness', 0.5965034953447873),
 ('linear', 0.545441794867133),
 ('recovery', 0.5145082758656173),
 ('state', 0.46520479583853164),
 ('hygiene', 0.45619050515495274),
 ('pandemic', 0.45059546190290467),
 ('coughing', 0.4438132433871827),
 ('decision', 0.31884111881348587),
 ('hand', 0.3101333487251366),
 ('interventions', 0.3075476026023653),
 ('applying', 0.2560174774597946),
 ('intensity', 0.24501529984427917),
 ('deaths', 0.22985041845545628),


In [21]:
l2 = top(comparative_scores, 'environmental', 30)

In [22]:
list(zip(list(zip(*l1))[0], list(zip(*l2))[0]))

[('environmental', 'adjust'),
 ('cues', 'greater'),
 ('fcv', 'changes'),
 ('survival', 'viruses'),
 ('disasters', 'transmission'),
 ('factors', 'environmental')]

In [23]:
top(comparative_scores, 'neurological', 30)

[]

In [17]:
top(comparative_scores, 'mers')

[('individuals', 0.17212272793241304),
 ('p', 0.1614529646770799),
 ('fatality', 0.1334063585626627),
 ('ed', 0.13167464833949932),
 ('mers', 0.1104246551227619),
 ('identify', 0.09265019832072449),
 ('perceived', 0.09099253819678416),
 ('search', 0.08933853575913712),
 ('participants', 0.08933853575913712),
 ('risk', 0.08905988349646546)]

In [18]:
top(comparative_scores, 'fatality')

[('mers', 0.4980965888121998),
 ('air', 0.25359821376488123),
 ('incubation', 0.23734529708392899),
 ('comorbidity', 0.22122346614920416),
 ('cases', 0.21046864604492416),
 ('comorbidities', 0.205240924198479),
 ('pollution', 0.205240924198479),
 ('sars', 0.19830044005021039),
 ('coronaviruses', 0.1894069705270051),
 ('difference', 0.17373223608561106)]

In [99]:
seeds = ('transmission', 'spillover',
 'transmissions',
 'spread',
 'spreading',
 'dispersal',
 'transmissibility',
 'transmitting',
 'transmitted',
 'importation',
 'contact',
 'contagion',
 'dissemination',
 'fomite',
 'reintroduction',
 'outbreaks',
 'epidemic',
 'emergence',
 'movement',
 'epidemics',
 'introductions')


seeds = ('immunities', 'immunity',
 'responses',
 'immune',
 'humoral',
 'defenses',
 'angioplasticity',
 'response',
 'protection',
 'defense',
 'immunopathology',
 'autoimmunity')
# seeds = ('eef1a', )

seeds = ('comorbidities',)

from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))


word_related = defaultdict(int)
for seed in ['economic_impact']:
    for word, score in sorted(scores.get(seed, {}).items(), key=lambda x: -x[1])[:30]:
        if word in stopwords: continue
        word_related[word] += score
    
    
sorted(word_related.items(), key=lambda x: -x[1])

[('economic', 12.649368678901082),
 ('impact', 4.433679419253568),
 ('gdp', 3.3477161136735605),
 ('billion', 3.0224013274860733),
 ('societal', 3.004155748785821),
 ('industry', 1.857920677108869),
 ('tremendous', 1.5350503155381203),
 ('morbidity', 1.0383317469931643),
 ('industries', 0.8442261963388724),
 ('worldwide', 0.6436443902605244),
 ('substantial', 0.6435282508902141),
 ('costs', 0.609372143568077),
 ('livestock', 0.5307305632567649),
 ('productivity', 0.5173617997292473),
 ('poultry', 0.4984051036682616),
 ('bank', 0.3651700779638117),
 ('burden', 0.350624306056061),
 ('swine', 0.33930164934531754),
 ('cattle', 0.30522338960748274),
 ('mortality', 0.2863867262934438),
 ('estimated', 0.2690911781820873),
 ('million', 0.24125701799432703),
 ('significant', 0.23632951105267866),
 ('countries', 0.22315846447140136),
 ('losses', 0.22306242401936183),
 ('cost', 0.21894960145090603),
 ('huge', 0.21785068514040476),
 ('estimates', 0.21636221197907418),
 ('cause', 0.2127689754464602

In [106]:
impact_papers = []

for paper in covid_papers:
    for field, value in data.iter_fields(paper, flat=True):
        for i, t in enumerate(value['tokens'][:-1]):
            if t == 'economic' and value['tokens'][i + 1] == 'impact':
                impact_papers.append(paper)
                break

In [107]:
len(impact_papers)

17

In [108]:
from importlib import reload

reload(snippets)
snps = snippets.get_snippets(impact_papers, word_related)

In [109]:
snps.sort(key=lambda x: -x['score'])

In [110]:
for snippet in snps[:20]:
    snippets.dm(snippet['snippet'])
    snippets.dm('**keywords**: {}'.format(', '.join(snippet['queries'])))
    snippets.dm('---')

The global macroeconomic **impact** of SARS was **estimated** at USD 30-100 **billion** or around USD 3-10 **million** per cases [14] . The 2003 SARS outbreak caused **losses** of USD 12.3-28.4 **billion** and an **estimated** decrease of 1% in **GDP** in China and 0.5% in Southeast Asia [15] . The social **burden** of SARS in Guangzhou meant less income and spending, with a rough estimate of the total **economic** burden of RMB 11 **billion** [16] .

**keywords**: estimated, impact, losses, economic, million, burden, billion, gdp

---

Influenza virus is a negative sense, segmented RNA virus that presents a **substantial** burden to human health. Despite the availability of successful vaccines and antivirals, infection with seasonal influenza viruses is still the **cause** of 3-5 **million** cases of severe illnesses and up to 300-650 thousand **deaths** worldwide (World Health Organization, 2017) . In the US alone, it is **estimated** that seasonal influenza affects up to 9 **million** people a year resulting in 12,000-56,000 **deaths** annually (Centers for Disease Control and Prevention, 2018c) . The **economic** burden is **estimated** to be in the tens of billions of dollars (Putri et al., 2018) . In addition, pandemic influenza outbreaks emerge at unpredictable intervals causing increased **morbidity**, **mortality**, and a negative **economic** impact (Erbelding et al., 2018) .

**keywords**: cause, deaths, estimated, morbidity, economic, million, substantial, mortality

...

**keywords**: industry, economic, losses

---

By 26 January also, almost 50 **million** people in Wuhan and neighbouring cities had effectively been placed in quarantine while the WHO had determined that the event should not yet be declared as a Public Health Emergency of International Concern (PHEIC) [2] and had recommended no specific travel restrictions. The WHO have emphasised the importance of exit screening at ports in **countries** showing transmission of the novel coronavirus and have provided guidance for **countries** implementing entry screening at airports while acknowledging that evidence for the effectiveness of entry screening is equivocal.

**keywords**: million, countries

---

In [64]:
sent = 'exponential growth rate (estimated at 0.14 [11] )'

In [93]:
from collections import defaultdict
import re

er_pattern = re.compile('growth.{,20}?rate.{,30}(?P<rate>\d\.\d+)', re.I)

matching_papers = defaultdict(set)
matching_snippets = []

for pid, paper in enumerate(covid_papers):
    for field, value in data.iter_fields(paper, flat=True):
        for match in er_pattern.finditer(value['text']):
            matching_papers[float(match.groupdict()['rate'])].add(pid)
            matching_snippets.append((paper['metadata.title'][0]['text'], value))

In [98]:
for title, s in matching_snippets:
    snippets.dm(f'**{title}**')
    snippets.dm(re.sub('(rate)', '**\g<0>**', s['text']))
    snippets.dm('---')

**Clinical Medicine Assessing the Impact of Reduced Travel on Exportation Dynamics of Novel Coronavirus Infection (COVID-19)**

Here, we consider the median time to a major epidemic in (8) and (9). Since an exponential growth of cases has been observed, we let the hazard be an exponential function. Then, the integral of the hazard function holds the form: C · (exp(rt) − 1), where C is a constant (assumed to be one for the following calculation), and r is the exponential growth **rate** estimated at 0.14 per day [16] . The doubling time is then calculated as t d = ln(2)/r = 4.95 days. The difference in the median date between (8) and (9) is thus described as: (2) .

---

**Clinical Medicine Incubation Period and Other Epidemiological Characteristics of 2019 Novel Coronavirus Infections with Right Truncation: A Statistical Analysis of Publicly Available Case Data**

Here, r is the exponential growth **rate** (estimated at 0.14 [11] ), T is the latest time of observation (31 January 2020), and F(.) is the cumulative density function of f (.).

...

where r is the exponential growth **rate** estimated at 0.14 [5] and T is the latest 10 time of observation (12 February 2020). The widely applicable information 11 criterion (WAIC) was used to compare between distributions and the model with 12 the minimal WAIC value was selected as the best-fit model for each set of 13 estimates with and without right truncation. 14

---

**Assessing the impact of reduced travel on exportation dynamics of novel coronavirus infection (COVID-19)**

where C is a constant (assumed to be one for the following calculation), and r is the exponential growth **rate** estimated at 0.14 per day [15] . The doubling time is then calculated as t d =ln(2)/r=4.95 days. The difference in the median date between (8) and (9) is thus described as:

---

In [81]:
for pid in d[0.14]:
    print(covid_papers[pid]['metadata.title'][0]['text'])

Clinical Medicine Incubation Period and Other Epidemiological Characteristics of 2019 Novel Coronavirus Infections with Right Truncation: A Statistical Analysis of Publicly Available Case Data
Journal Pre-proof Serial interval of novel coronavirus (COVID-19) infections Title: Serial interval of novel coronavirus (COVID-19) infections Authors
Title: Serial interval of novel coronavirus (COVID-19) infections Correspondence to
Assessing the impact of reduced travel on exportation dynamics of novel coronavirus infection (COVID-19)
Title: The Novel Coronavirus, 2019-nCoV, is Highly Contagious and More Infectious Than Initially Estimated
Incubation Period and Other Epidemiological Characteristics of 2019 Novel Coronavirus Infections with Right Truncation: A Statistical Analysis of Publicly Available Case Data
Clinical Medicine Assessing the Impact of Reduced Travel on Exportation Dynamics of Novel Coronavirus Infection (COVID-19)


In [68]:
by_seed = defaultdict(list)

for snippet in snps:
    key = f"{snippet['lgd']['word']}...{snippet['lgd']['seed']}"
    by_gd[key].append(snippet['sent'])
            
        if snippet['rgd']:# and not stop_snippet(snippet['rgd']): 
            key = f"{snippet['rgd']['seed']}...{snippet['rgd']['word']}"
            by_gd[key].append(snippet['sent'])

IndentationError: unexpected indent (<ipython-input-68-3c9e630ae444>, line 7)

In [70]:
[e.close() for e in tqdm._instances.copy()]

 35%|███▍      | 444/1278 [10:14<19:13,  1.38s/it]


[None]

In [73]:
for k, sents in sorted(by_gd.items(), key=lambda x: -len(x[1]))[:50]:
    sents = '\n  - ' + ('\n\n\n  - '.join(sents))
    dm(f"""
<details>
  <summary> <strong> {k} </strong> </summary>
  {sents}
</details>
    """)



<details>
  <summary> <strong> hypertension...diabetes </strong> </summary>
  
  - Third, although aging impacts immunity to viral infection [30] , patients older than 60 years of age usually have some **comorbidities, such as hypertension, diabetes,** or chronic bronchitis, and seek medical attention only when these primary diseases worsen when combined with ARI.


  - **hypertension and diabetes)** or risk of infectious diseases among pregnant women.


  - The enrolled patients had various **comorbidities; three patients were complicated with anxiety and depression requiring treatment, sixteen had hypertension, five had diabetes,** seven had peptic ulcers, and one had Sjogren's syndrome.


  - Underlying medical conditions in these patients included, in descending order of frequency, **hypertension, diabetes** mellitus, chronic lung diseases, cerebrovascular disorders, cardiovascular disorders, malignancy, pregnancy, chronic hepatic diseases, chronic renal diseases, and neuromuscular diseases.


  - In the multivariate analysis, **cardiovascular disorders (OR 4.05, 95% CI 1.72-9.50; P = 0.001), chronic lung diseases (OR 3.38, 95% CI 1.51-7.61; P = 0.003), hypertension (OR 2.37, 95% CI 1.26-4.47; P = 0.008), diabetes** mellitus (OR 3.09, 95% CI 1.57-6.06; P = 0.001), and neuromuscular diseases (OR 10.18, 95% CI 1.30-79.40; P = 0.027) were independently associated with hospitalization of patients with laboratory-confirmed influenza (Table 5 ).


  - We were also able to confirm that underlying chronic diseases such as **cardiovascular disorders, chronic lung diseases, hypertension, diabetes** mellitus, and neuromuscular diseases were important factors in determining hospitalization of patients with laboratory-confirmed influenza.


  - Patients with **comorbiditie**s (hypertension,** diabe**tes, chronic lung diseases, cardiovascular disorders or neuromuscular diseases) were at greater risk of hospitalization after influenza viral infection.


  - A SI measurement ≥1 was associated with lower positive likelihood ratios in patients ≥65 years, with **hypertension, diabetes** or using β-/Ca 2+ channel-blockers (range 4.9 to 6.5) compared with patients not exposed to these factors (range 7.6 to 11.6).


  - Co-morbidities consisted of **hypertension (29.5%), renal insufficiency (13.6%), diabetes** (9.1%), dyslipidemia (5.7%), cardiac diseases (4.5%) and liver diseases (4.5%).


  - Except asthmatic subgroup, **comorbidities such as hypertension, diabetes** mellitus, hyperlipidemia and coronary artery disease were frequently observed.


  - Our first case is a 69-year-old Caucasian man with a past medical history of **coronary artery disease, chronic renal insufficiency, hypertension and type 1 diabetes.**


  - Our second case is a 71-year-old Caucasian woman with a past medical history significant for **hypertension, type 1 diabetes** and chronic hepatitis C.


  - The risk of developing type-2 **diabetes (T2D) increases with age, obesity, cardiovascular diseas**e (hypertension,** dyslipidaemia), lack of physical activity, and family history of diabe**tes.


  - The signaling complex РKС/Nox is activated at different pathophysiological states: neurodegenerative diseases [120] , **atherosclerosis [121] , hypertension [122] , diabetes** [123] , and cancer [124] .


  - Concomitant medical complications included: **hypertension (n = 5), diabetes** mellitus (DM)(n = 1) and asthma (n = 1).


  - Several studies have verified that congenital heart disease has several influence factors, such as increased maternal age, prenatal infections, gestational **hypertension, gestational diabetes,** etc.


  - Therefore, we should provide pre-pregnancy education to avoid gestational **hypertension and diabetes** and improve prenatal diagnosis rate to adopt measures in advance.


  - It is explained by the high rate of comorbidity among our patients (more than half with arterial **hypertension and diabetes** mellitus), the deep immunosuppression required and the numerous complications of lung grafting.


  - A total of 26 cases with fatalities were used to disclose the dangerous **comorbidities, showing that the major comorbidities are hypertension (53.8%), diabetes** (42.3%), coronary heart disease (19.2%), cerebral infarction (15.4%), chronic bronchitis (19.2%) and Parkinson's disease (7.7%) (Figure 2 ).


  - Though the symptom compositions reported for the confirmed cases are similar to that of the fatality cases we collected, the percentages of **hypertension, diabetes,** and coronary heart diseases are much higher among the fatality cases than among the


  - From the analysis of the fatal cases of this novel coronavirus pneumonia, the **comorbidities of hypertension, diabetes,** coronary heart disease, cerebral infarction, and chronic bronchitis were found to be dangerous factors that resulted in death.


  - Patient medical records were reviewed and clinical data were extracted, including demographic characteristics, body mass index (BMI), smoking history, diagnostic method, Sequential Organ Failure Assessment (SOFA) score, nosocomial infection, and **comorbiditie**s (hypertension,** diabe**tes mellitus, chronic obstructive pulmonary disease [COPD], chronic kidney disease, chronic liver disease, current or previous solid organ malignancy).


  - With respect to chronic diseases in poor patients, e.g., **hypertension, diabetes,** tuberculosis and severe mental disorders, family doctors were paid to offer systematic health management.


  - Based on the involvement of PRCP in the conversion of these peptide hormones, the enzyme has also been associated with diseases, such as **hypertension, diabetes** mellitus, obesity, inflammation, and cardiovascular dysfunction (90, 91).


  - Antiretroviral therapy for HIV may increase the risk of developing metabolic syndrome (abdominal **obesity, hyperglycaemia, dyslipidaemia and hypertension) and thus predispose to type 2 diabetes** and cardiovascular disease.


  - Results from this study showed that the most common medical condition among the pilgrims was **hypertension (17.9%) followed by diabetes** (7.7%).


  - Patients were at significant CV risk with 69% having a smoking history, and a history of CAD in 66%, dyslipidaemia in 60%, **hypertension in 82%, diabetes** in 24% and AF in 11%.


  - The prevalence of CAD and cardiac risk factors including dyslipidaemia, **hypertension, diabetes** and cigarette smoking were similar in the two groups, as was LVEF <50%, the use of pharmacological agents, low density lipoprotein cholesterol, triglycerides levels, kidney function and troponin level (all p > 0.05).


  - Health practices may differ significantly by group, particularly in health-promotion strategies and approaches to disease screening between source and host nations (for example, **hypertension, diabetes** in pregnancy, colon, uterine or breast cancer).


  - In humans, CRF is mainly caused by **hypertension, glomerulonephritis, and diabetes** mellitus 3, 4 .


  - However, in two of the papers studied the indices had been developed in populations with one condition only, either **hypertension 23 or type 2 diabetes.**


  - In 2009, China launched the NEPHSP with nine items, including health records establishment, health education, immunization, child health, maternal health, geriatric health, **hypertension and type 2 diabetes** management, severe mental illness management, and the surveillance and control of infectious diseases and public health emergencies.


  - Standard management of **hypertension and diabetes** has reached 86.27 million and 24.19 million patients respectively, in an equivalent to management rates of 35% and 30%.


  - As to outcome indicators for systematic care of patients with **hypertension and diabetes,** the mortality of ischemic stroke and ischemic heart diseases increased in 2000-2013, except the mortality of haemorrhagic stroke has decreased since 2005, and mortality of diabetes increased slightly since 2005 (Fig.


  - Cardiovascular risk assessment comprised age, sex, **hypertension, hypercholesterinaemia, diabetes** mellitus or ischaemic cardiomyopathy ( Table 1) .


  - Currently, commercially available synthetic protease inhibitors targeting host proteases include (1) thrombin and Factor Xa inhibitors for coagulation (such as dabigatran and rivaroxaban); (2) angiotensin converting enzyme (ACE) inhibitors for **hypertension (such as lisinopril); (3) DPP-4 inhibitors for diabetes** (such as sitagliptin); and (4) proteasome inhibitors for multiple myeloma (such as bortezomib) ( Figure 2 ).


  - Dabigatran and Rivaroxaban are anti-coagulants; Lisinopril is for **hypertension; Sitagliptin is for diabetes;** and Bortezomib is for multiple myeloma.


  - Coexisting conditions including **coronary heart disease, chronic obstructive pulmonary disease, hypertension, diabetes,** cerebrovascular disease and cancer.


  - According to the data from the third and fourth China National Health Services Survey, Chinese adults had worse health status in terms of prevalence of chronic diseases in 2008 than that in 2003, particularly **hypertension, diabetes,** heart disease and stroke.


  - To improve the public health care, the Ministry of Health instituted eleven basic items of public health services in 2009, including the establishment of residents' health assessments, health education, health care of children, maternity patients, elderly and patients suffering from **hypertension, type II diabetes** and severe psychosis, vaccine inoculation and immunization, response to public health emergencies and assistance in sanitary control [16] .


  - Although some village clinics had the authentic information about patients who suffered from **hypertension and diabetes,** they reported that a large proportion of patients were reluctant to see a doctor in the clinics because they were dissatisfied with the medical conditions there.


  - The contract service there is a series of pre-paid package services divided into different kinds of chronic diseases including **hypertension, diabetes,** severe psychosis, chronic obstructive pulmonary disease, malignant tumor and disability.


  - According to the statistics collected in March 2014, 76.50% of the contracted families bought **hypertension and/or diabetes** packages charging in middle class.


  - However, more than 90% patients suffering from **hypertension and diabetes** in the pilot villages we visited, had been controlled by the village clinics, according to the county health bureau's report.


  - County B also applied an audacious way to supplement the funding for public health, which became the most crucial incentive mechanism and underpinned the effectiveness in **hypertension and diabetes** control.


  - Distinctively, county B applied an innovative fund raising way and partially solved the problem, also made an initial step toward **hypertension and diabetes** control.


  - Inflammation in response to comorbid conditions, such as **hypertension and diabetes,** may play a proportionally larger role in HFPEF as compared to HF with reduced ejection fraction (HFREF).


  - TNFa, TNFR1 and TNFR2 were each significantly associated with at least two of the following: age, estimated glomerular filtration rate, **hypertension, diabetes,** smoking, peripheral vascular disease or history of atrial fibrillation.


  - Inclusion criteria for healthy controls were as follows: no history of **cardiovascular or renal disease, hypertension, diabetes,** or atrial fibrillation; and no prescriptions for antiarrhythmics; ACEi; ARB; beta-blockers; digoxin; loop or thiazide diuretics; or MRA.


  - We explored the relationship between age, eGFR, sex, **obesity, LVH, hypertension, diabetes,** smoking status, peripheral vascular disease or history of atrial fibrillation with TNFa, TNFR1, TNFR2 and IL-6 in the combined HF group (Table 3) .


  - TNFa, TNFR1 and TNFR2 increased differentially in response to smoking status or comorbid conditions, such as **hypertension and diabetes,** but were always associated with at least two covariates, while IL-6 was not associated with any covariate (Table 3) .


  - The subjects in this study were 50 healthy controls (age .18 yrs), recruited from the hospital or research centre staff, with no history of kidney disease, **hypertension, or diabetes,** and 100 renal transplant recipients from The Ottawa Hospital Renal Transplant Program, age .18 yrs, and .3 months posttransplant.


  - After adjusting for potential confounding factors [age, gender, **diabetes, eGFR, albuminuria, hypertension, and use of calcineurin inhibitors] only diabetes** was significantly associated with urinary ACE2 activity (p = 0.003) and protein levels (p,0.001, Table 2 ) in transplant patients.


  - His past medical history was significant for **hypertension, diabetes** mellitus, chronic kidney disease, and non-Hodgkin's lymphoma with receipt of an allogeneic stem cell transplant 13 years prior.


  - Failure to translate effective therapeutic strategies from the 'bench to bedside' may partly be attributed to the use of animal models that do not incorporate non-modifiable risk factors such as gender and many of the **stroke comorbidities observed in the clinical stroke population, such as hypertension, diabetes,** obesity, etc.


  - There is growing evidence that the function of several types of vascular K + channels is altered during major **cardiovascular diseases, such as chronic hypertension, diabetes,** and atherosclerosis [40] .


  - Furthermore, one of the pivotal hallmarks of **cardiovascular and metabolic diseases, including hypertension, diabetes,** atherosclerosis, stroke, and heart failure, is reduction-oxidation (redox) imbalance, a condition, known as oxidative stress [17] .


  - Among the ESRD affected participants, **hypertension (61.7%) was the most common comorbidity, followed by type 2 diabetes** mellitus (41.2%).


  - For the 2019-nCoV, limited comorbidity data is available; however, the twenty-six patients that have succumbed to the novel CoV had significant health conditions including **hypertension, diabetes,** heart and/or kidney function issues that may have made them more susceptible.


  - For the MERS-CoV outbreak, smoking, **hypertension, diabetes,** cardiovascular disease, and/or other chronic illnesses have been present in the majority of deaths and correspond to findings in animal models [25] .


  - The most common **comorbidities were hypertension in 54(32%) patients, cardiac failure in 36(21.3%) situations and diabetes** in 32(18.9%) cases.


  - His past medical history was significant for **hypertension and diabetes** type 2.


  - After this, a 61-year-old Saudi male with **hypertension and diabetes** with no history of smoking, reported for surgery.


  - Physical inactivity has been linked to **obesity, coronary hearth disease, hypertension, strokes, diabetes,** colon cancer, breast cancer and osteoporotic fractures [49] .


  - The fourth adult patient (S1), while not obviously immunosuppressed, also suffered from liver cirrhosis, **hypertension, type 2 diabetes,** and co-infection with herpes simplex virus, and required mechanical ventilation.


  - In addition, most of those who have died from MERS-CoV, SARS-CoV and 2019-nCoV were advance in age and had underlying health conditions such as **hypertension, diabetes** or cardiovascular disease that compromised their immune systems [114] .


  - In addition, to assess the physical and mental status of the participants, self-rated health status, chronic disease status **(hypertension, diabetes)** and daily life stress level were surveyed (S2 and S3 Files).


  - In this survey, if respondents have **hypertension or diabetes,** they categorized in 'Chronic disease present'.


  - Totally, 17 participants claimed to be healthy and not suffering from chronic conditions, whereas the remaining 23 participants suffered from various chronic conditions, including **hypertension, diabetes** mellitus, gout, liver disease, and heart disease.


  - Consequently, it has a beneficial role in many diseases such as **hypertension, diabetes,** and cardiovascular disease where its expression is decreased.


  - Its actions appear critical in a variety of disease states, including **hypertension, diabetes,** ageing, renal impairment, and cardiovascular disease.


  - Second, chronic diseases in 2008 were more highly prevalent than in 2003, particularly **hypertension, diabetes,** heart disease, and stroke.


  - For example, the prevalence of **hypertension and diabetes** in 2008 approximately doubled since 2003, and that of stroke increased by 1.4 times.


  - Therefore a national education program that can eliminate the huge gaps among presence, awareness, treatment, and ability to control of **hypertension and diabetes** should be given to the public, clinicians, and healthcare decision makers.


  - Especially, prevalence of **hypertension and diabetes** in 2008 was twice that in 2003.


  - s=Serum leptin concentrations and leptin resistance, are independent markers of **obesity, weight gain, systemic hypertension [101] , diabetes** mellitus [102] , obstructive sleep apnoea [103] and myocardial infarction [104] , while polymorphisms of the leptin gene are associated with insulin resistance [105] and long-term risk of developing diabetes mellitus [102] .


  - Cardiovascular disease, the leading cause of premature death and disability in most western countries, has a wellestablished multi-factorial basis involving a complex interplay between genetic predisposition, environmental and personal risk factors -including systemic **hypertension, diabetes** mellitus, hyperlipidaemia, obesity and cigarette smoking -and more recently recognized mechanisms, including endothelial dysfunction [118] , vascular inflammation [119] and leptin levels [104] .


  - Systemic **hypertension, diabetes** mellitus and hyperlipidaemia have long-established, but complex, patterns of inheritance, a situation further compounded by evidence receptor polymorphisms -including those of angiotensin II type 1 receptor [120] , IRS-1 gene [121] and low density lipoprotein receptor (LDLR) [122] -both confer disease susceptibility and have regionally variable prevalences [123, 124] .


  - This hypothesis exists at the confluence of established risk factors for **coronary artery disease, including genetic susceptibility, polymorphisms predisposing to hypertension [126] [127] [128] , diabetes** [126] and hypercholesterolaemia and substantial new data implicating vascular inflammation [119, 129] , endothelial dysfunction [119, 130] , leptin dysregulation [104] and viral infection [131, 132] in the pathogenesis of vascular disease.


  - Buraczynska found that an association between renalase gene polymorphism and **hypertension in type 2 diabetes** [32] .


  - Our results suggested that history of smoking and/or underlying clinical conditions may increase risk for infection, especially for patients with **hypertension, diabetes,** and heart-pulmonary insufficiency.


  - Nearly half of the patients (8/18) had **hypertension, two had diabetes,** three had coronary heart diseases, and three had COPD, indicating that chronic heart and lung diseases would increase risk for H7N9 infection.


  - [5, 6] In addition, an elevation of SUA level has been shown to be associated with **hypertension, [7] diabetes** mellitus (DM), metabolic syndrome, coronary disease, [8] and even cardiovascular mortality.


  - ICD-9-CM codes were also used to identify patients treated with IABP who underwent PCI or CABG, and to identify patients treated with IABP who had baseline **comorbidities of hypertension (HTN) (ICD-9-CM: 401-405), diabetes** mellitus (DM) (ICD-9-CM: 250), hyperlipidemia (ICD-9-CM: 272), myocardial infarction (MI) (ICD-9-CM: 410), and cerebrovascular attack (CVA) (ICD-9-CM: 430-438).


  - A 61-year-old male, who was originally from KSA and had a history of **hypertension and diabetes** without smoking, is reported here.


  - 10 Furthermore, the increased maternal age is closely related to the occurrence of gestational **hypertension and gestational diabetes,** all of which are risk factors for congenital heart disease.


  - Obesity and overweight (i.e., BMI> 25 kg/m 2 , prevalence: 24.5%, 95% CI: 18.6-31.6%), **hypertension (17.1%, 13.3-21.8%) and diabetes** (13.3%, 9.3-18.8%) were the most prevalent comorbidities in DENV.


  - However, **hypertension (45.0%, 39.1-51.0%), diabetes** (24.7%, 20.2-29.8%) and heart diseases (25.6%, 19.5-32.7%) were the most prevalent in WNV.


  - Obesity/overweight was the most prevalent comorbidity in DENV patients (24.5%, 95% CI: 18.6-31.6%), followed by **hypertension (17.1%, 95% CI: 13.3-21.8%) and diabetes** (13.3%, 95% CI: 9.3-18.8%) (Fig 2) .


  - On the other hand, in WNV cases, **hypertension was the most frequent comorbidity (45.0%, 95% CI: 39.1-51.0%), followed by heart diseases (~25%), diabetes** (~25%) and stroke (10.1%, 95% CI: 7.1-14.3%) (Fig 3) .


  - The present study demonstrates that **hypertension and diabetes** are the most prevalent comorbidity in both DENV and WNV with obesity/overweight and heart diseases being present, respectively, in about 20% of the DENV and WNV cases.


  - Seven (26%) patients had at least one medical comorbidity **(hypertension [n = 6], diabetes** mellitus [n = 2] and asthma [n = 2]); and nine (33%) were obese (body mass index ≥ 27 kg/m 2 [35] ).


  - Similarly, most of the pilgrims are older people (>50 years) with **comorbidities such as hypertension, diabetes,** and liver diseases [5, 6] .


  - The patient had been receiving routine-biannual health check-ups, and did not have any major disease, except **hypertension and diabetes** mellitus.


  - He had been under clinidipin and sitagliptin treatments for 5 years for **hypertension and diabetes,** respectively.


  - Co-morbidities included **hypertension, diabetes** mellitus, cerebral infarction, coronary artery disease, and these co-morbidities were in stable state which was confirmed by physicians from related departments.


  - Of the 122 cases (47%) with co-morbid conditions, the most common were **hypertension (76 cases, 29%), diabetes** (72 cases, 28%), heart disease (47 cases, 18%), obesity (41 cases, 16%), smoking (41 cases, 16%), chronic renal disease (39 cases, 15%), immunecompromised status (24 cases, 9%), chronic lung disease (19 cases, 7%), steroid use (19 cases, 7%), asthma (12 cases, 5%), chronic liver disease (8 cases, 3%), cancer (5 cases, 2%), pregnancy (5 cases, 2%), chronic hematologic disease (2 cases, 0.7%), systemic lupus erythematosus (2 cases, 0.7%), eclampsia (1 case, 0.4%), and hypothyroid (1 case, 0.4%).


  - The most frequent **comorbidities recorded were hypertension (43.6%), psychiatric disorders including anxiety and/or depression (42.0%), heart failure (32.8%), ischemic heart disease (28.3%), and diabetes** mellitus (28.0%).


  - Potentially confounding factors were assessed as well, including age, education, smoking status (never or ever smoker), alcohol drinking status (non-drinker or drinker), depression score (Center for Epidemiologic Studies-Depression Scale, CES-D), and selfreported comorbidity **(hypertension, diabetes,** stroke, and arthritis).


  - After adjusting for age, education, smoking, alcohol drinking, depression score, comorbidity **(hypertension, diabetes,** stroke, and arthritis), and communication frequency, the men with lower density (<0.71) and higher size (4-6) had a higher BMI (β=1.089, p=0.037) than the men with higher density (>0.83) and lower size (1-2), but this was not the case in the women (β=0.391, p=0.393).


  - We adjusted for **comorbidities, such as hypertension, diabetes,** stroke, and arthritis, in order to overcome this unintended effect.


  - He was taking medications for **hypertension and diabetes** mellitus.


  - His past medical history included **hypertension, type 2 diabetes** mellitus, hypercholesterolemia, severe coronary triple vessel disease, ischemic cardiomyopathy, and left ventricular dysfunction.


  - The fifth topic of "lifestyle disease" contained the following words: "test", "study", "patient", "risk", **"hypertension", "result", "diabetes"**, "cholesterol", "research team", "hepatitis", "male", "brother", "heart disease", "death", "swine flu", "USA", "stroke", "abnormal", "professor", "colon cancer".


  - Several studies 9, 11, [16] [17] [18] [19] including the current study (Table 3 , suggest that the male gender; health care worker; MERS patients above the age of 60 with an underlying medical conditions, such as **hypertension, diabetes** and renal failure; those who had exposure to camels and camel milk consumption; and close contact with MERS-CoV cases had a significantly higher risk of severe disease, including death.


  - A national public health policy which is being implemented aims to improve the competence of primary health workers in the care of people with chronic diseases such as **hypertension, diabetes** and psychoses (Ministry of Health, 2009) .


  - Personal data included sex, age, epidemiological history, history of smoking, and **comorbidities [e.g., chronic obstructive pulmonary disease (COPD), cancer, hypertension and/or diabetes]**.


  - There was no significant difference in any comorbidity including **hypertension, diabetes,** chronic obstructive pulmonary disease (COPD), cancer, and others between the two groups (all P > 0.05).


  - [14] Probably because of the small sample size, there was no significant difference in any comorbidity including **hypertension, diabetes,** COPD, cancer, and others between the two groups.


  - 27 Patients with chronic diseases such as **hypertension, heart disease, diabetes,** and obesity are more vulnerable to excessive temperatures and at risk of complications.


  - The reduction in NO due to the competition between Arg and iNOS is also important in the pathogenesis of a variety of diseases, including **atherosclerosis and hypertension, myocardial ischemia/reperfusion injury, diabetes,** allergic asthma and inflammatory diseases (47) (48) (49) (50) .


  - 16 This was based on inadequately concentrated urine and failure to identify other metabolic causes of **hypertension such as diabetes** mellitus or hyperthyroidism.


  - 13, 14) We estimated potential-outcome means (POMs) and average treatment effect (ATE) in the general population, patients with **hypertension, and those with diabetes** mellitus (DM) using an augmented inverse-probability-weighted (AIPW) estimator that combined elements of regression adjustment on PS and PS weighting estimation among several PS analysis methods.


  - The types of medical institutions were determined by assessing which institution the participants used from among clinics, hospitals, and tertiary hospitals if they had a chronic disease such as **hypertension or diabetes** mellitus.


  - Continuity of care, accessibility, and patient-centeredness, which are vital in the treatment of chronic diseases such as **hypertension and diabetes** mellitus, are primary health care characteristics that positively influence health outcomes and health care costs.


  - **hypertension, diabetes,** epilepsy, hypotiroidism, etc.)


  - In addition, most of those who have died had underlying health conditions such as **hypertension, diabetes** or cardiovascular disease that compromised their immune systems.


  - The aims of this study were to determine prevalence of **hypertension in feline diabetes** mellitus (DM) and its potential relation with different parameters (age, body condition score (BCS), BCS previous to diagnosis, sex, glycemic control, time since diagnosis, concurrent diseases, chronic corticoid treatments, diabetic neuropathy and remission).


  - Of the 28 patients, 10 (35.7%) had at least 1 underlying disease including **hypertension, diabetes,** asthma, chronic rhinitis, dyslipidemia, and hypothyroidism, and 1 patient had undergone surgery for lung cancer (Table 1) .


  - Chen et al [2] and Huang et al [4] also reported that 32-51% of the hospitalized patients with confirmed COVID-19 had **hypertension, diabetes, and cardiovascular diseases, while 10 of the 28 Korean patients (35.7%) had at least 1 underlying disease, including hypertension** and diabetes.


  - *Any underlying medical condition including chronic lung disease (asthma, chronic obstructive **pulmonary disease, interstitial lung disease, and bronchiectasis), chronic heart disease (coronary artery disease or congestive heart failure, hypertension, and valvulopathy), diabetes** mellitus, chronic kidney disease (with or without dialysis), neurologic disorders (epilepsy, cerebral palsy, dementia, or history of stroke), and chronic liver disease (hepatitis, cirrhosis, or hepatic failure).


  - These patients often had underlying diseases such as **hypertension, diabetes,** and coronary heart diseases, with some having multiple underlying diseases.


  - We collected well-known risk factors for hematoma expansion from patients' data by using electric medical records and the PACS system: patient's age and sex, location of hematoma, level of consciousness, systolic BP at admission, medical history (e.g., **hypertension, diabetes,** dyslipidemia, cardiovascular disease and chronic kidney disease [CKD]), alcohol consumption, smoking status, use of an antiplatelet agent, use of anticoagulants, and hematoma volumes.


  - There was no statistically significant difference between the 2 groups in mean age, sex, size and volume of hematoma, spot signs on hematomas, BP, medical history **(hypertension, diabetes** mellitus, dyslipidemia, cardiovascular disease, stroke, alcohol), and drug use (antiplatelet or anticoagulant), except smoking status and CKD.


  - [10] Concomitant diseases included arterial **hypertension (n = 5), diabetes** mellitus (n = 3), hypercholesterolemia (n = 2), coronary heart disease (n = 2), sleep apnea (n = 2), and celiac disease (n = 2).


  - The most common coexisting medical conditions were **hypertension (31.7%), diabetes** (18.8%), solid organ malignancy (13.4%), and chronic lung disease (10.2%).


  - 18 In the Fukuoka ischemic **stroke registry, mean age was 70 years, 76% had hypertension, 27% diabetes** mellitus, 46% hyperlipidemia, and 26% atrial fibrillation.


  - Experience with HIV/AIDS treatment scale-up (40) could provide useful lessons for a standardized approach to management of **hypertension and diabetes,** thereby enhancing cost-effectiveness; facilitating supervision, monitoring, and evaluation; and ensuring accountability.


  - Fifteen patients had **hypertension, 5 had diabetes** mellitus, 1 patient was HIV infected and 1 had congenital kidney disease.


  - Three had asthma, five had **hypertension, and one had diabetes** mellitus type 2.


  - The ratio of **hypertension, diabetes,** earlier myocardial infarction and ACBG were 58%, 27%, 32% and 15%, respectively, in the monitored population (13 females, 28 males; mean age: 68 years).


  - The presence of microalbuminuria (MA) is a known risk factor in patients with **hypertension and diabetes.**


  - Smoking, high cholesterol, **hypertension, and diabetes** did not significantly differ between the groups.


  - Risk factors including **hypertension and diabetes** were documented.


  - • all persons aged ≥50 years, because in this group greatly increases the number of people belonging to high risk groups • adults and children who have chronic **pulmonary(including asthma), or cardiovascular (except isolated hypertension), renal, hepatic, neurological, hematologic or metabolic disorders (including diabetes** mellitus);


  - All case-patients had concurrent medical conditions; the most common were heart disease (70%, 14/20), dementia (65%, 13/20), **hypertension (40%, 8/20), diabetes** (35%, 7/20), and lung disease (35%, 7/20).


  - 37.2% of patients had at least one underlying disorder (i.e., **hypertension, diabetes,** cardiovascular diseases and chronic lung diseases).


  - Of these deceased patients, 64.9% had at least one underlying disorder (i.e., **hypertension, diabetes,** cardiovascular disease, or chronic obstructive pulmonary disease) ( Table 2 ).


  - 57.0% of the patients had at least one of the concomitant diseases including **hypertension, diabetes,** cardiovascular diseases and chronic lung diseases.


  - Meanwhile, numbers of patients with COVID-19 infection had chronic **comorbidities, mainly hypertension, diabetes** and cardiovascular disease, which is similar to MERS-COV population.


  - Seventy-one patients had underlying diseases, including 28 patients with **hypertension, 12 patients with diabetes,** and 31 patients with other conditions.


  - The most common chronic medical diseases were **hypertension (n =12, 34.3%) and diabetes** (n =7, 20%).


  - All the six AMI patients were aged >60 years; five of them had two or more underlying **comorbiditie**s (hypertension,** diabe**tes, cardiovascular diseases, and chronic obstructive pulmonary disease).


  - Patients who progressed to severe NCP were more likely to have underlying **hypertension (P = 0.004), but did not otherwise have significant differences in other co-morbidities including diabetes,** chronic obstructive pulmonary disease (COPD), cardiovascular disease, cerebrovascular disease and chronic hepatitis B infection.


  - All of those who died had underlying diseases, the most common of which was **hypertension (16/25, 64%), followed by diabetes** (10/25, 40%), heart diseases (8/25, 32%), kidney diseases (5/25, 20%), cerebral infarction (4/25, 16%), chronic obstructive pulmonary disease (COPD, 2/25, 8%), malignant tumors (2/25, 8%) and acute pancreatitis (1/25, 4%).


  - Interpretation The age and underlying diseases **(hypertension, diabetes,** etc.)


  - As concerning the underlying diseases, the most common one was **hypertension, followed by diabetes,** Heart disease, kidney disease, cerebral infarction, COPD, malignant tumors and acute pancreatitis.


  - In conclusion, the age and underlying diseases **(hypertension, diabetes,** etc.)


  - The analysis of COVID-19 patients with cardiac-related chronic diseases **(coronary artery diseas**e, hypertension** and type 2 diabe**tes mellitus) showed that they were more prone to be in the severe or critical conditions.


  - Percentage of patients with **hypertension was higher in severe group (22 [52.4%] vs 4 [12.9%], P < 0.001), while other coexisting disorders, including diabetes,** cardiovascular disease and chronic obstructive pulmonary disease showed no significant differences.


  - Thirty-three severe patients with a median age of 66 years(range,33-82 years,IQR, 57-76) were treated in intensive care units; out of these patients, 66.7 %(22) were men and 19 (57.5%) had chronic diseases, including **hypertension, diabetes,** heart failure, stroke, and renal insufficiency.


  - Including 33 patients in the intensive care unit (ICU), the median age of the patients was 66 years(IQR, 57-76), ranging from 33-82 years old, with 22 (66.7%) being men and 19 (57.5%) patients having chronic diseases, including **hypertension, diabetes,** heart failure, stroke, and renal insufficiency.


  - The bulk of death cases had comorbidity (76.8%), including **hypertension (56.1%), heart disease (20.7%), diabetes** (18.3%), cerebrovascular disease (12.2%), and cancer (7.3%).


  - https://doi.org/10.1101/2020.02.26.20028191 doi: medRxiv preprint of death cases had comorbidity (75.6%), including **hypertension (56.1%), heart disease (20.7%), diabetes** (18.3%), cerebrovascular disease (12.2%), and cancer (7.3%).


  - 16 Moreover, we found underlying diseases such as **hypertension, heart disease and diabetes** were very common in our death cases, and 30.5% of patients had 2 or more comorbidities.


  - Consistent with recent reports, our results suggest that 2019-nCoV is more likely to infect elderly people with chronic **comorbidities as a result of the weaker immune functions of these patients [5, 6] , and 2019-nCoV-associated death is also related to elder age and underlying illnesses, especially hypertension, diabetes,** and cardiac disease.


  - There were 10 patients with basic diseases (27.78%), including 7 patients with **hypertension (19.44%), 4 patients with diabetes** (11.11%), 1 patient with hypothyroidism (2.78%), and 1 patient with hyperlipidemia (2.78%).


  - Thirty-nine cases (38.2%) had **comorbidities, including hypertension, diabetes,** coronary artery disease, tuberculosis, hepatitis B and chronic bronchitis.


  - The majority of our patients were 50 to 70 years old with common chronic diseases, such as **hypertension, diabetes,** tuberculosis, hepatitis B, chronic bronchitis, etc.


  - Participants were assigned a '1' if they endorsed a history of each of the following medical conditions: **cerebrovascular risk factors including hypertension, heart failure, COPD, anemia, diabetes;** endocrine dysfunction including thyroid disease, testosterone therapy, estrogen therapy; kidney disease.


  - Thus, our main goal is to establish the odds ratio for COVID-19 mortality, adjusted to age and gender-specific prevalence of the morbidities reported in the CDC paper 11 , namely **hypertension, diabetes,** cardiovascular disease, chronic respiratory disease and cancer.


  - Age is described by intervals of decades (0-9; 10-19; up to ≥ 80 years old) and the **comorbidities are: hypertension, diabetes,** cardiovascular disease, chronic respiratory disease and cancer.
</details>
    


<details>
  <summary> <strong> diabetes...hypertension </strong> </summary>
  
  - Exclusion criteria for the patients and the control subjects were history of a chronic disease (tuberculosis, **diabetes, hypertension,** heart disease, hepatic diseases, etc.


  - We focused on the most common chronic diseases that were set as nation health priorities for treatment and prevention, such as **diabetes, heart disease, asthma, hypertension,** and cancer.


  - A 57-year-old patient presenting with type 2 **diabetes mellitus and hypertension** was admitted to Saint-Pierre Hospital with fever (39 • C), an elevated C-reactive protein (248 mg/L), and acute urinary retention requiring the placement of a urinary catheter.


  - Of the 79 hospitalized patients, the frequency of **diabetes, hypertension,** cases having at least one of the comorbid conditions, and the proportion of elderly were significantly higher in patients with influenza A (H3N2) (P,0.05).


  - Because old age, **diabetes, essential hypertension,** and β-/Ca 2+ channel-blockers might reduce the compensatory increase in heart rate and mask blood pressure reductions in shock or pre-shock states, we hypothesized that these factors weaken the association between SI and mortality, reducing the utility of SI to identify patients at risk.


  - We examined whether age ≥65 years, **diabetes, essential hypertension,** and use of β-/Ca 2+ channel-blockers modifi ed the association between SI and mortality.


  - The risk of developing type-2 **diabetes (T2D) increases with age, obesity, cardiovascular diseas**e (hypertension,** dyslipidaemia), lack of physical activity, and family history of diabe**tes.


  - Allergy was not evaluated because the symptoms are very broad to specifically define as an allergy, and there would be significant potential information bias as it is less likely to clinically diagnosed allergy as compared with asthma, **diabetes, hypertension** and heart disease.


  - We identified older age, male sex and underlying medical conditions, including **diabetes mellitus, renal disease, respiratory disease, heart disease and hypertension,** as clinical predictors of death associated with MERS.


  - Several clinical predictors of death were identified including older age, male sex and underlying medical conditions, including **diabetes mellitus, renal disease, respiratory disease, heart disease and hypertension.**


  - Exclusion criteria were ischemic heart disease, respiratory, renal or neurological disease, **diabetes mellitus, hypertension,** BMI > 30, BMI < 18; smoker; pregnant; use of vasoactive medication or NSAIDS/aspirin within 48 h of study; current involvement in other research studies.


  - The chemical chaperone TUDCA is a classic ER stress inhibitor that improves ER folding capacity, which has been protective in various diseases, including **diabetes mellitus, hypertension,** calcification, and even cardiac dysfunction by preventing ER stress (135) (136) (137) .


  - The comparison of probability of malaria infection was made between patients with **diabetes (n=675) and a control group (n=791) comprising patients attending hypertension** clinics, patients attending other hospital clinics, and hospital staff.


  - The one person in the general population group who was seropositive for MERS-CoV was female in her mid-50s with **diabetes and hypertension.**


  - Other studies have shown no change in blood pressure in ACE2 knock-out (KO) mice compared to control C57BL/6 (Tikellis et al., 2008; Shiota et al., 2010) , but once **diabetes was induced in ACE2 KO mice, hypertension** did develop (Tikellis et al., 2008) .


  - However, in our own study of 503 Australian Caucasians with type 2 **diabetes , the prevalence of hypertension** was significantly higher in both men and women and was associated with the G allele of the ACE2 SNP rs4240157 .


  - reported that, among 41 confirmed cases, the symptom composition is as follows: pneumonia (100%), fever (98%), cough (76%), lymphopenia (63%), Dyspnea (55%), fatigue (44%), sputum production (28%), headache (8%), hemoptysis (5%), and diarrhea (3%); and the underlying diseases included **diabetes (20%), hypertension (15%), and cardiovascular** disease (15%) [21] .


  - Hypoalbuminaemia is also an important marker of severe infection and/or byproduct of chronic disease such as **diabetes, hypertension** and CKD.


  - Seventeen patients had **diabetes (33.3 %), 25 had hypertension (49 %), 14 (27.5 %) had end stage renal disease, eight (15.7 %) had coronary** artery disease and six (11.8 %) patients were immunosuppressed, two of whom were pregnant.


  - Analysis of the clinical features of 41 hospitalized patients with laboratory-confirmed 2019-nCoV infection revealed that 30 were men (73%); less than one-half had underlying co-morbid conditions (13; 32%) which included **diabetes (8, 20%) , hypertension (6, 15%), and cardiovascular** disease (6; 15%); and the average age was 49.0 years old.


  - All of the normal control mothers, which had no pregnancy-related problems, such as **diabetes or hypertension,** had or were carrying fetuses lacking any congenital defects.


  - Until more is understood about MERS-CoV infection, we believe that patients with **diabetes mellitus, hypertension,** ischemic heart disease, and end-stage renal diseases should be considered at a higher risk for MERS-CoV infection and in the triage, these patients seem to be prioritized for treatment.


  - First, long-term treatment with GCs is often accompanied by severe side effects, such as **diabetes, increased risk of infection, osteoporosis, hypertension,** and so forth [25, 26] .


  - Linagliptin has also demonstrated efficacy and tolerability in type 2 **diabetes patients with hypertension** and microalbuminuria [149] .


  - The study authors collected various clinical data: fever (temperature ≥ 38°C); presence of any of the following respiratory symptoms: cough, bloody cough, shortness of breath, or chest pain; presence any of the following gastrointestinal symptoms: diarrhea, vomiting, or nausea; MERS-CoV infection; intensive care unit (ICU) admission; hospital: KAMC-Riyadh or KFGH-Jeddah; abnormal radiology findings; **diabetes; renal disease; and hypertension.**


  - Alterations in the composition of the gut microbiota and reductions in microbial diversity lead to different disorders such as inflammatory conditions of the intestine (inflammatory bowel disease, irritable bowel syndrome, colorectal cancer) (Gagnière et al., 2016; Ghoshal et al., 2017; Rapozo et al., 2017) , type 2 **diabetes, obesity, anorexia nervosa, forms of severe acute malnutrition, cardiovascular diseases (atherosclerosis, hypertension,** heart failure) (Tang et al., 2017) , neurobehavioral diseases (autism spectrum disorder, major depression) (Clemente et al., 2012) .


  - The newest study including the first 41 confirmed cases admitted to Wuhan between 16 December 2019 and 2 January 2020 showed the median age of patients was 49 years; and the main underlying diseases were **diabetes, hypertension, and cardiovascular** diseases.


  - Age above 65 years, history of dengue infection, **diabetes mellitus, hypertension** and renal insufficiency have been identified as being significantly associated with DHF/DSS (Lee et al., 2006) .


  - The one person in the general population group who was seropositive for MERS-CoV was female in her mid-50s with **diabetes and hypertension.**


  - In fact, the contract service policy aims to construct a closer relationship between rural residents and village doctors, which can urge village doctors to improve the efficacy of public health services, for example, the early detection of **diabetes and hypertension,** and other chronic diseases control [23] .


  - Twenty-eight (24.3%) of 115 patients of had at least one underling illness, including **diabetes, ischemic heart disease, hypertension, chronic obstructive pulmonary** disease, asthma, cerebrovascular disease, and chronic hepatitis.


  - The tree shows that age is the most discriminatory attribute, followed by BMI, family history of **diabetes, work-related stress, physical activity, salty food preference, hypertension, gender, and history of cardiovascular** disease or stroke.


  - 1 Including: **diabetes, hypertension, cardiovascular,** neurological, and immunodeficiency diseases; 2 This was calculated as collection date minus admission date; 3 Included: cephalosporin, piperacillin/tazobactam, and meropenem; 4 Included: fluconazole, voriconazole, and liposomal amphotericin B.


  - Overactivation of the RAS in **diabetes, especially Ang II and ACE [3] , leads to hypertension,** fluid retention, and inflammation, causing renal and vascular end-stage disease in the long term [35] .


  - ACE2 activity) while the following explanatory variables were entered into the model: age, gender, **diabetes, eGFR, albuminuria, hypertension,** and use of calcineurin inhibitors.


  - The median age for transplant subjects was 54 (IQR, 42-62) yrs, 65% were male, 38% had **diabetes, and 88% had hypertension.**


  - After adjusting for potential confounding factors [age, gender, **diabetes, eGFR, albuminuria, hypertension, and use of calcineurin inhibitors] only diabetes** was significantly associated with urinary ACE2 activity (p = 0.003) and protein levels (p,0.001, Table 2 ) in transplant patients.


  - Some studies suggest that **diabetes, hypertension,** chronic heart disease, and chronic kidney disease are risk factors for MERC-CoV infection and a severe clinical presentation [19] ; however, this theory needs to be further investigated.


  - Both groups were matched regarding the age, gender, body mass index, the association of **diabetes or hypertension,** and Euro score.


  - The AKI group match the other group with respect to age, gender, body mass index, the association of **diabetes or hypertension.**


  - **diabetes, hypertension,** chronic respiratory disease, etc.)


  - Recognized risk factors for IVIG-induced thrombosis Frontiers in Immunology | Primary Immunodeficiencies include male gender; age >60; **diabetes; renal insufficiency, dyslipidemia; hypertension; immobility; coronary** disease; pre-existing vascular disease, family history of early thromboembolic disease; atrial fibrillation, high-dose and high-speed IVIG infusions.


  - The comorbidity conditions that might have effect on disease susceptibility, such as **diabetes mellitus, heart disease, hypertension,** asthma, tuberculosis, and malignancy, were traced by reviewing medical records and self reporting.


  - They had no history of cardiorespiratory illness, influenza-like illness in the week prior to the trial, or any pregnancyrelated complications such as gestational **diabetes, hypertension,** intrauterine growth restriction, placenta praevia, ruptured membranes, or threatened preterm labor.


  - Case report A 60 year old man with a history of **diabetes and hypertension** was referred to our clinic with the suspicion of infective encephalitis.


  - According to the pre-embarkation medical assessment, 28% of pilgrims were classified as high risk due to underlying health conditions such as **diabetes, hypertension,** other chronic diseases or if they were 60 years or older.


  - The covariate list included the following: gestational age at first antenatal visit, maternal age less than 19 y, maternal age more than 35 y, multiple births, maternal medical risk factors, labor/delivery complications, birth defects, maternal **diabetes (gestational and/or non-gestational), hypertension** (treated or untreated), mother insured, multivitamin use in pregnancy, history of smoking during pregnancy, history of alcohol use during pregnancy, black race, education less than 12 th grade, mother's marital status, and maternal weight before pregnancy.


  - b In the secondary adjusted models, the covariates included gestational age at first antenatal visit, maternal age less than 19 y, maternal age more than 35 y, multiple births, maternal medical risk factors, labor/delivery complications, birth defects, maternal **diabetes (gestational and/or non-gestational), hypertension** (treated or untreated), mother insured, multivitamin use in pregnancy, history of smoking during pregnancy, history of alcohol use during pregnancy, black race, education less than 12th grade, mother's marital status, and maternal weight before pregnancy.


  - b In the secondary adjusted models, the covariates included gestational age at first antenatal visit, maternal age less than 19 y, maternal age more than 35 y, multiple births, maternal medical risk factors, labor/delivery complications, birth defects, maternal **diabetes (gestational and/or non-gestational), hypertension** (treated or untreated), mother insured, multivitamin use in pregnancy, history of smoking during pregnancy, history of alcohol use during pregnancy, black race, education less than 12th grade, mother's marital status, and maternal weight before pregnancy.


  - Altered G6PD activity is associated with pathophysiology, such as autophagy, insulin resistance, infection, inflammation, as well as **diabetes and hypertension.**


  - ), pre-existing diseases such **diabetes or hypertension,** or choice of treatment [26, 30] .


  - The covariates were age (in years), sex (male, female), nationality (Saudi, Non-Saudi), admission year (2014-2015, 2016-2017), vital signs on admission (body temperature, heart rate, respiratory rate, systolic and diastolic blood pressure), MERS-CoV associated conditions on admission (pneumonia/acute respiratory distress syndrome, septic shock/multiple organ failure, abdominal pain/diarrhoea), pre-existing comorbid conditions on admission **(diabetes, hypertension,** hepatitis C, chronic renal failure, and chronic heart disease), receipt of the treatment regimen (yes, no), duration of hospital stay (in days), status at discharge (alive, dead).


  - Variables that showed a significant association (P <0.10) in univariate analysis (age, sex, the dialysis type, BMI, SGA, albumin level, TA-SUA, and **diabetes) or were of considerable theoretical relevance (total Kt/V, calcium level, phosphorus level, hypertension,** gout, uricosuric drugs, and antiuricosuric drugs) were entered into the multivariate stepwise linear regression models.


  - 1. any known cardiorespiratory conditions (including but not limited to the following: asthma, bronchitis, cystic fibrosis, congenital heart disease, emphysema) 2. any known medical conditions that may be exacerbated by strenuous physical activity, including but not limited to the following: exercise-induced asthma, lower respiratory infections (including pneumonia, bronchitis) in the past 2 weeks, anxiety disorders, **diabetes, hypertension,** or epilepsy/seizure disorders 3. any physical disability from medical, orthopaedic or neuromuscular disorders 4. have an acute upper respiratory tract infection or symptomatic rhinitis (i.e.


  - MERS-CoV may infect people of any age, but older age, underlying comorbidity (such as **diabetes mellitus, renal disease, respiratory disease, heart disease, and hypertension)**, and delayed confirmation or late diagnosis are all factors that affect MERS disease outcomes and mortality [2] [3] [4] [5] [6] [7] .


  - The OR of severe WNV was 4.21 in patients with **diabetes (95% CI: 2.22-7.92) and 2.72 in those with hypertension** (95% CI: 1.78-4.14) (Table 5 ).


  - Although the prevalence of **diabetes, hypertension** and heart diseases varied widely among the DENV selected studies (ranging from 60-to 100-fold), the vast majority of the reports showed values clustering around the pooled estimated averages for each comorbidity, as evidenced by the low I 2 index values.


  - In line with these findings, the results of the present study strongly suggest that **diabetes, heart diseases and hypertension** are more prevalent in severe DENV and WNV cases than in the non-severe disease and may present significant risk factors-together with advanced age-in complication of flavivirus infection.


  - We provided evidence for a higher prevalence of **diabetes, hypertension,** and heart diseases in severe cases of flavivirus infections such as DENV and WNV than in nonsevere cases.


  - Most of the patients had no major comorbidity, except one had **diabetes mellitus and the other two had hypertension.**


  - CHD and **diabetes were the commonest medical risk factors reported among H7N9 patients, and the prevalence of smoking and hypertension** was higher in subjects with H7N9 compared with the other patient groups.


  - Moreover, red ginseng is believed to improve immune function, physical performance and sexual function, and manage cancer, **diabetes (DM) and hypertension.**


  - The second topic of "lifestyle disease" contained the following words: **"diabetes", "osteoporosis", "hypertension", "exercise", "risk", "cholesterol", "obesity"**, "blood vessels", "blood pressure", "metabolic syndrome", "abnormal", "stroke", "gout", "bone", "study", "fat", "intake", "smoking", "weight", and "fracture".


  - Healthy staff members, excluding those having underlying high-risk illnesses such as **diabetes, chronic lung, heart (except for hypertension)**, renal, and liver diseases, as well as those taking immunosuppressive drugs and pregnant women are recommended to be assigned to related task areas (BII) [39, 40] .


  - The prevalence of **diabetes, hypertension,** and chronic kidney disease (CKD) was 13.3%, 13.3%, and 10%, respectively.


  - The prevalence of **diabetes, hypertension,** and CKD was not different between the two groups.


  - Baseline characteristics of the 3 patients who underwent CRRT were as follows: #1, 71-yr old male who had **diabetes, hypertension, CKD, and heart disease; #2, 76-yr old male who had CKD, chronic pulmonary** disease, and chronic liver disease; #3, 86-yr old male who had diabetes, hypertension, and heart disease.


  - The prevalence of **diabetes, hypertension,** and renal disease were 67%, 50%, and 42%, respectively.


  - She had underlying **diabetes mellitus and hypertension,** and was diagnosed with atri-37 al fibrillation 7 days prior to admission.


  - (a) General treatment principles: treatment according to the symptoms and the underlying disease **(diabetes, hypertension,** etc.


  - Other chronic conditions have been reported in 96% of clinical cases including **diabetes, hypertension,** heart disease and kidney disease.


  - The relationship between **diabetes mellitus (DM) and hypertension** is complex.


  - Hypertension was not significantly correlated with the control of DM; however, none of the dogs considered as in good control of the **diabetes had severe hypertension** (P = 0.067).


  - As with most chronic diseases, such as **diabetes and hypertension,** constant maintenance and management are required.


  - Moreover, the percentage of patients with **diabetes (87.5% vs 43.75%), hypertension** (75% vs 18.75%) as well as those on mechanical ventilation (25% vs 0%), interferon and ribavirin (25% vs 0%), oseltamivir (50% vs.6.25%), and those with chronic kidney disease (37.5% vs 6.25%), and cancer (25% vs 6.25%) were higher among those cases compared to their counterparts with medical costs lower than the mean cost.


  - 31 Most of the patients in the study sample had chronic health conditions, such as **diabetes (58%), hypertension** (37%), chronic kidney disease (17%), and cancer (12%).


  - Uncontrolled **diabetes mellitus and hypertension were the most common comorbidities** reported among three of the cases.


  - Uncontrolled **diabetes mellitus, hypertension, and ischemic heart diseases were the most common comorbidities** present and were reported among three cases including the deceased.


  - Finally, Saudis live longer now, which explains the rise in cancer cases and chronic illnesses like **diabetes, hypertension and hyperlipidemia.**


  - Comorbid medical conditions were documented for 88 (62%) cases, with **diabetes (n = 63; 44%) and hypertension** (n = 57; 40%) most common.


  - 39 Results emanating from different studies have shown that influenza outbreaks are characterized by high severity of symptoms with increased mortality, 6, 8, 13, 25, 32 Several factors have been associated with H1N1 disease severity, such as factors due to the virus (ie, viral pathogenic mutations, resistance to antivirals), factors inherent to host susceptibility (eg, age, sex, race), including physiological immunosuppression or acquired diseases (ie, **diabetes, hypertension, obesity,** asthma), factors associated with available medical services and public health facilities, and factors arising from the presence of bacterial coinfections.


  - Patient B had no underlying disease such as **diabetes, hypertension** or chronic renal disease.


  - This striking difference in the incidence of renal impairment is likely due to the low prevalence of underlying conditions such as chronic renal diseases, **diabetes and hypertension** in our population.


  - He had a history of atrial fibrillation, **diabetes mellitus, hypertension,** chronic kidney disease, and hypothyroidism.


  - AD genes are also expressed in the blood-brain barrier (BBB), which is disrupted by AD risk factors (age, alcohol, aluminum, concussion, cerebral hypoperfusion, **diabetes, homocysteine, hypercholesterolemia, hypertension, obesity,** pesticides, pollution, physical inactivity, sleep disruption, smoking) and by pathogens, directly or via olfactory routes to basal-forebrain BBB control centers.


  - These include **diabetes, midlife hypertension or obesity,** smoking, and physical inactivity [206] .


  - HAC dogs were at increased risk for concurrent **diabetes mellitus, urinary tract infection, urolithiasis, hypertension,** gall bladder mucocele, and thromboembolic disease compared with dogs without HAC.


  - In traditional Chinese medicine, it has been used in treatments for hyperlipemia, **diabetes mellitus, Alzheimer's disease, acute liver injury, inflammation, photophobia, headache, dizziness and hypertension** (4) (5) (6) .


  - A 74-year-old Saudi male patient with **diabetes, hypertension, and dyslipidemia** presented with a 3 days of ataxia, vomiting, confusion, and fever of 39.5 °C (Fig.


  - Ten participants had coexisting conditions **(diabetes mellitus, hypertension,** asthma).


  - One additional patient had **diabetes mellitus type 2 and hypertension.**


  - Among the 42 sporadic case-patients, median age was 57 (range 25-89) years; 35 (83%) were male, and 33 (79%) reported underlying conditions, most commonly **diabetes (n = 24) and hypertension** (n = 23).


  - Median patient age was 56 (range 29-84) years; 28 (88%) were male, and 23 (72%) reported underlying conditions, most commonly **diabetes (n = 19) and hypertension** (n = 15).


  - Compared with NICUGP, patients in ICUGP were more likely to have developed postoperative arrhythmia (57.9% vs 12.9%, P < 0.001), were older, of higher ASA status, and more likely to have **diabetes, coronary artery disease, hypertension,** a higher cancer stage, and to have received more intraoperative blood products.


  - Sixteen percent (16/88) of the infected patients reported underlying conditions such as human immunodeficiency virus (HIV) and human T-cell leukemia virus (HTLV)-type infection, **diabetes mellitus, liver transplant, megacolon syndrome, hypertension,** Wiskott-Aldrich syndrome, and congenital kidney and lung diseases.


  - It is governmentally funded offering free screening for breast cancer, **diabetes, hypertension and obesity** for all women above 45 years.


  - Six (13%) HCWs reported having a chronic medical condition (e.g., asthma, **diabetes, hypertension)**, and 6 (13%) reported smoking tobacco.


  - A total of 49 participants (1 case-patient and 48 non-case-patients) reported having >1 chronic condition (e.g., asthma, **diabetes, heart disease, hypertension,** breast cancer) ( Table 1 ).


  - The MERS case-patient reported having asthma; among non-case-patients, the most common chronic conditions reported were asthma (31%), **diabetes (25%), and hypertension** (18%).


  - Certain underlying conditions, including **diabetes mellitus (DM), hypertension,** chronic cardiac disease, and chronic renal disease, are recognized risk factors for illness and death caused by infection with MERS-CoV (2,3).


  - The first case-patient was a 65-year-old male retiree with a history of **diabetes, heart disease, smoking, and hypertension.**


  - The third case-patient was a 76-year-old male farmer with a history of **diabetes, heart disease, and hypertension.**


  - The fifth case was a 73-year-old male retiree with **diabetes and hypertension.**


  - In addition, she also suffered from type 2 **diabetes mellitus, hypertension,** osteoporosis, bilateral osteoarthritis of the knees, and an old traumatic fracture of the left humeral neck.


  - All but two patients (96%) had one or more chronic medical conditions, including **diabetes (68%), hypertension** (34%), heart disease (28%), and kidney disease (49%).


  - 2 patients had **diabetes and hypertension** respectively.


  - 24 Similarly, in our study, all the six AMI patients were aged >60 years and had one or more underlying conditions, including **diabetes, hypertension, COPD, and cardiovascular** diseases.


  - Epidemiological survey shows that COVID-19 patients with underlined conditions such as **diabetes, hypertension, cardiovascular** disease or the elderly are not only susceptible, but also often more serious.


  - There was no significant difference of **diabetes, cardiovascular disease and chronic obstructive pulmonary disease between the two groups, but the patients with hypertension** in the severe group were more than those in the mild group (p = 0.056) ( Table 1) .


  - After adjusting for age and smoking status, patients with COPD (HR 2.681, 95%CI 1.424-5.048), **diabetes (HR 1.59, 95%CI 1.03-2.45), hypertension** (HR 1.58, 95%CI 1.07-2.32) and malignancy (HR 3.50, 95%CI 1.60-7.64) were more likely to reach to the composite endpoints than those without.


  - On admission, 20-51% of patients reported as having at least one comorbidity, with **diabetes (10-20%), hypertension (10-15%) and cardiovascular** and cerebrovascular diseases (7-40%) being most common [3, 4, 6] .


  - For instance, **diabetes [29] and COPD [30] frequently co-exist with hypertension or coronary** heart diseases.


  - Among three deaths, one was infected with SARS-CoV-2 after recent lung cancer surgery, one had **diabetes, ischemic cardiomyopathy, and hyperthyroidism, and one had hypertension and coronary** heart disease [16, 17] .
</details>
    


<details>
  <summary> <strong> pulmonary...hypertension </strong> </summary>
  
  - In addition to their role in vascular diseases, chymases were also found to be associated with lung diseases such as **pulmonary fibrosis, pulmonary arterial hypertension** (PAH), asthma, and COPD [116, [120] [121] [122] .


  - For example, aldosterone has been implicated in the pathogenesis of **pulmonary hypertension [62] , and spironolactone has been shown to attenuate experimental pulmonary hypertension** via MR inhibition in pulmonary artery smooth muscle cells [63] .Admittedly, bleomycin is also a frequently used tool drug to induce pulmonary hypertension [64, 65] .


  - Attenuation or lack of CD39 activity is associated with vascular dysfunction and remodeling in **pulmonary arterial hypertension** [32] , decreased hepatic regeneration, and failure of vascular reconstitution [33] .


  - Expression was undetectable in lung from patients with **pulmonary arterial hypertension.**


  - Apelin is down-regulated in **pulmonary arterial hypertension (PAH), a devastating disease characterized by vascular remodeling resulting in progressive obliteration of the pulmonary** circulation, leading to right ventricle (RV) hypertrophy and right heart failure (Alastalo et al., 2011; Chandra et al., 2011) .


  - The importance of BMP signaling for the vascular system is demonstrated by the association of mutations in genes encoding for BMPRII and ALK1 with the development of two genetic vascular diseases, namely, **pulmonary arterial hypertension** and hereditary hemorrhagic telangiectasia.


  - Also, in **pulmonary arterial hypertension virus infection is considered to be a possible risk factor (203) , and pulmonary** hypertension has been reported to be a side effect of prolonged treatment with type I IFN (204, 205) .


  - TRAIL has been found to be increased within **pulmonary vascular lesions of patients with pulmonary hypertension (206) and also in a mouse model of hypoxia-induced pulmonary** hypertension, levels of soluble TRAIL correlated with right ventricular systolic pressure, right ventricular hypertrophy, and pathologic alterations (33, 34) .


  - MiRNAs have shown to have an effect also upon **pulmonary arterial hypertension** (PAH).


  - [78] , where AntagomiR-223 was efficient in reducing levels of miR-223 in **pulmonary artery and lungs of rats as compared to controls, but failed to attenuate pulmonary hypertension.**


  - shown that hypoxia results in reduced nitric oxide levels, a principle cause of **pulmonary vasoconstriction and that inhalation of nitric oxide can reduce hypoxic pulmonary hypertension,** improve the patient's ventilation to perfusion ratio, improve hypoxemia, and improve HAPE in patients with signs and symptoms, as well as decrease hospitalization time with few adverse effects.


  - Anisodamine can improve **pulmonary vascular spasms, decrease pulmonary vascular resistance and hypoxic pulmonary hypertension, improve the pulmonary** microcirculation, and maintain smooth pulmonary blood flow.


  - cystic fibrosis, **pulmonary sarcoidosis, pulmonary hypertension f All solid tumors including bronchial carcinoma g Patients with one of the following conditions: primary or secondary antibody deficiency, congenital immunodeficiency, immunosuppressive therapy other than steroids, severe malnutrition with cachexia h Highest/lowest value within a time period of 3 days before and 3 days after date of rtPCR i Other infections (n = 10); neoplastic diseases (n =6); collagen vascular; other rheumatologic or autoimmune (n = 14); sarcoidosis (n = 3); non-infectious nonneoplastic pulmonary** diseases (n = 23); cardiovascular diseases (n = 3) Other chronic lung diseases, e.g.


  - cystic fibrosis, **pulmonary sarcoidosis, pulmonary hypertension** f All solid tumors including bronchial carcinoma g Patients with one of the following conditions: primary or secondary antibody deficiency, congenital immunodeficiency, immunosuppressive therapy other than steroids, severe malnutrition with cachexia h Highest/lowest value within a time period of 3 days before and 3 days after date of rtPCR i Other infections (n = 14); neoplastic diseases (n = 1); collagen vascular; other rheumatologic or autoimmune (n = 1); gastroesophageal diseases (n = 1) Other chronic lung diseases, e.g.


  - cystic fibrosis, **pulmonary sarcoidosis, pulmonary hypertension f All solid tumors including bronchial carcinoma g Patients with one of the following conditions: primary or secondary antibody deficiency, congenital immunodeficiency, immunosuppressive therapy other than steroids, severe malnutrition with cachexia h Highest/lowest value within a time period of 3 days before and 3 days after date of rtPCR i Other infections (n = 24); neoplastic diseases (n = 7); collagen vascular; other rheumatologic or autoimmune (n = 15); sarcoidosis (n = 3); non-infectious nonneoplastic pulmonary** diseases (n = 23); cardiovascular diseases (n = 3); gastroesophageal diseases (n = 1) Pathogens associated with mixed infections were counted individually 31 of 40 adults (78%).


  - (2) neurological diseases (Alzheimer's disease, Parkinson's disease, ischemia, central nerve system injury): a disintegrin and metalloproteinase domain-containing protein 10 (ADAM10), γ-secretase, β-secretase 1, and MMP-24; (3) inflammation (arthritis, chronic obstructive **pulmonary disease (COPD), asthma, Crohn's disease, inflammatory bowel disease, colitis, diverticulitis, chronic liver disease): MMPs, cathepsins, neutrophil elastase, and ADAM17; (4) blood coagulation: thrombin, factor Xa; (5) diabetes: dipeptidyl peptidase 4 (DPP4); (6) cardiovascular disease (atherosclerosis, hypertension, cardiomyopathy, congestive** heart failure, myocardial infarction, neovascularization and cardiac remodeling, cardiac fibrosis, aneurism, ischemia-reperfusion injury): angiotensin converting enzyme, renin, MMPs, chymase, neprilysin, calpain; (7) osteoporosis: cathepsin K; and (8) autoimmune diseases (multiple sclerosis, lupus, Guillain-Barre syndrome, psoriasis): activated protein C, kallikreins; ageing (skin), MMPs, neutrophil elastase [1, 2] .


  - Chronic lung tissue changes and **pulmonary alveolar proteinosis (PAP), as well as pulmonary arterial hypertension,** have been described in adult patients [1, [7] [8] [9] .


  - PAP was found in 18% and **pulmonary arterial hypertension** in 9% of patients.


  - In dogs, heart disease has been related to serious conditions, including progressive heart failure, **pulmonary edema and/or congestion, hypertension, left arterial** rupture, and pericardial tamponade (Borgarelli et al., 2008; Guglielmini, 2003; Pouchelon et al., 2015) .


  - Acute lung injury (ALI) is a syndrome characterized by diffuse **pulmonary edema and severe hypoxemia in the absence of clinical evidence of left atrial hypertension** [1] .


  - [14] found an incidence of **pulmonary hypertension of 92.2% but did not find any association between pulmonary hypertension** and death.


  - Clinically, right ventricular failure has no agreed definition, but criteria (using **pulmonary artery catheter data) include pulmonary hypertension** associated with an RV cardiac index <2.5 L min −1 m −2 and a right atrial pressure >8 mmHg [9] .


  - Keywords: **pulmonary fibrosis, canine infectious respiratory disease complex (CIRDC), pulmonary hypertension,** pneumonia, interstitial lung disease, PIBO BACKGROUND Post-infectious bronchiolitis obliterans (PIBO), a syndrome in children most commonly caused by Mycoplasma pneumonia (1, 2) and adenovirus (1) and occasionally Bordetella pertussis (3, 4) is associated with chronic inflammatory and fibrotic lesions of small airways leading to chronic airflow obstruction (5) .


  - Another important cause of severe complications is **pulmonary edema, which can be consequence of left ventricular systolic or diastolic dysfunction, or due to the use of tocolytic agent, fluid overload, severe hypertension** or severe renal disease.


  - Vascular injury and remodeling may lead to **pulmonary arterial hypertension** which may compromise right ventricular function associated with a poor clinical outcome [13] .


  - Most hospitals and intensive care units worldwide use the standard criteria for the diagnosis of acute lung injury (ALI)/ARDS: presence of acute hypoxemia (PaO 2 /FIO 2 less than 300 mmHg or 39.99 Kpa for ALI or less than 200 mmHg or 26.66 Kpa for ARDS), bilateral infiltrates seen on a frontal chest radiograph that are consistent with **pulmonary edema, and no clinical evidence of left atrial hypertension, or (if it is measured) a pulmonary** artery wedge pressure (PAWP) of less than 18 mmHg according to the 1994-1998 American-European Consensus Conference on ARDS (AECC) [25, 26] .


  - (b) In cases of severe ARDS with right ventricular dysfunction and **pulmonary artery hypertension** consider prone position and inhaled nitric oxide.


  - IL-17 is also important for iBALT formation in patients with **pulmonary arterial hypertension** (92, 93) .


  - Chronic thromboembolic **pulmonary hypertension (CTEPH) is one of the primary causes of severe pulmonary hypertension.**


  - Pulmonary endarterectomy (PEA) can prevent mortality due to right ventricle failure (1) , which occurs in chronic thromboembolic **pulmonary hypertension (CTEPH), a type-4 pulmonary hypertension** (2) .


  - Mean **pulmonary artery pressure >25 mmHg at rest or >30 mmHg during exercise was considered to indicate the presence of pulmonary hypertension.**


  - The natural killer cell has been found to be involved in vascular remodeling, which may lead to **pulmonary arterial hypertension** (52, 53) .


  - Numerous studies have demonstrated that the polyamine regulatory pathway is a pharmacological target in **pulmonary arterial hypertension** (54, 55) .


  - Altered G6PD status is implicated in many cellular pathophysiological processes and diseases, including hypoxia, inflammation, microbial infection, sepsis, **pulmonary vessel dilation, diabetes, hypertension,** kidney disease, and brain injuries [24,30,.


  - Consequently, G6PD overactivation contributes to remodeling of **pulmonary arterial and development of pulmonary hypertension** [138] .


  - The severity of SNV-associated HCPS may have made it more apparent that the onset of **pulmonary edema, tachycardia and hypertension** seemed to be all but universally temporally associated with the appearance of a spectrum of highly-activated cells of the lymphoid lineage in the peripheral blood.


  - The indication for thrombolytic therapy was: (1) patients with haemodynamic instability or cardiogenic shock; (2) patients with massive **pulmonary artery obstruction (obstructions in more than 2 lobes on CTPA, or perfusion defects in more than 7 segments on V/Q scan) and combined right ventricular dysfunction (RVD) and pulmonary hypertension** on echocardiographic examination.


  - Using the current subset of data, five major classes of diseases were observed in the disease Ontology-based enrichment analysis as follows: cancer (prostate cancer, thyroid cancer, breast cancer and neoplasm metastasis), diseases of the respiratory or **pulmonary system (asthma, bronchial hyperreactivity, pulmonary alveolar proteinosis), degenerative diseases of the central nervous system (Amyotrophic lateral sclerosis, Parkinson's Disease), vascular disease (atherosclerosis, hypertension)** and encephalitis (rabies).


  - This process reduces **pulmonary ventilatory efficiency and leads progressively to general hypoxia and cardio-pulmonary hypertension** and failure.


  - The histological features of **pulmonary arteriopathy observed in 3 adult marmosets (0.47 %) were indicative of pulmonary hypertension,** and concurrent hypertrophic cardiomyopathy was present in at least one of the affected animals.


  - † Chronic lung disease included asthma, reactive airway disease, bronchopulmonary dysplasia, chronic obstructive **pulmonary disease, or emphysema, or the requirement for home oxygen combined with other lung conditions such as chronic respiratory failure or pulmonary hypertension.**


  - For example, apelin administration showed cardioprotective effects in heart failure 12 , and ameliorated the development of **pulmonary arterial hypertension** in rats 13,14 and humans 15 .


  - A previous study had used a concentration of up to 100 nmol/min for systemic infusion, where they obtained a therapeutic response in patients with **pulmonary arterial hypertension** and the highest dose was well tolerated 15 .


  - Furthermore, ER stress markers correlated with disease severity (the presence of **pulmonary arterial hypertension)** and IL-6 levels (260) .


  - ET A R/ET B R and selective ET A R antagonists, such as bosentan, macitentan, and ambrisentan, are used clinically for the treatment of **pulmonary arterial hypertension,** but their use is associated with side effects, including edema, anemia, increased risk of heart failure, and hepatic transaminitis (Wei et al., 2016; Packer et al., 2017) .


  - Based on the dual-feedback system linking the endothelin system and the RAS, dual AT 1 R/ET A R antagonists have been developed and tested in the clinic for **pulmonary arterial hypertension, essential hypertension,** and chronic kidney disease (Murugesan et al., 2002 (Murugesan et al., , 2005 Neutel et al., 2008; Komers and Plotkin, 2016; Komers et al., 2017) ; however, it is still to be established whether the protective benefits outweigh the risk of adverse reactions.


  - Therefore, recombinant human angiotensin-converting enzyme 2 (rhACE2) is currently considered for treating acute respiratory distress syndrome and **pulmonary arterial hypertension.**


  - The second study is an openlabel dose escalation study for rhACE2 (NCT03177603) in patients with **pulmonary arterial hypertension, in which changes in pulmonary** vascular resistance, cardiac output, and mean pulmonary artery pressure in response to rhACE2 serve as primary outcome measures.


  - Recombinant ACE2 and brain-selective APA inhibition are currently being tested clinically in **pulmonary arterial hypertension and overweight patients with hypertension,** respectively, based on the concept that ACE2 degrades Ang II and upregulates the protective Ang 1-7, whereas APA blockade prevents brain AT 1 R stimulation by Ang III.


  - In model 2, BNP was included as an additional variable in the regression model, since **pulmonary hypertension is associated with poor prognosis in chronic pulmonary tuberculosis 16, 17 and BNP is a non-invasive marker of pulmonary hypertension** 18, 19 (Table 3 ).


  - These findings are consistent with the results of previous studies 19, 25, 26 , which demonstrated that BNP levels were a prognostic factor in patients with idiopathic **pulmonary arterial hypertension, idiopathic pulmonary** fibrosis, and chronic cor pulmonale.


  - Even so, depending on severity, development of pre-capillary **pulmonary arterial hypertension** may also occur.


  - Unfortunately, right ventricular function in dogs with **pulmonary arterial hypertension** has been poorly investigated.


  - Venous thromboembolism (VTE) including acute **pulmonary embolism (APE), chronic thromboembolic pulmonary hypertension** (CTEPH) and deep venous thrombosis (DVT) is a global disease.


  - iNO therapy is currently FDA approved for the treatment of term and near-term neonates with hypoxemic respiratory failure associated with clinical or echocardiographic evidence of **pulmonary arterial hypertension** (26, 27) .


  - respiratory failure, **pulmonary heart disease cachexia) or COPD-non related (eg obesity, diabetes mellitus, arterial hypertension)** [23] [24] [25] .


  - Therefore, we evaluated the ability of transverse RV strain and strain rate, obtained by 2-dimensional speckle tracking echocardiography in healthy dogs and dogs with **pulmonary hypertension, to identify dogs with pulmonary hypertension.**


  - We classified dogs as having moderate **pulmonary hypertension if TR velocity > 3.5 m/sec, and severe pulmonary hypertension** if TR velocity > 4.5 m/sec.


  - Although transverse RV strain (but not strain rate) showed a negative relationship with class of **pulmonary hypertension, it was not useful in identifying dogs with pulmonary hypertension.**


  - In **pulmonary hypertension (PH) patients, this led to the reconceptualization of the RV as part of right ventricular pulmonic circulation unit and the acknowledgement that RV function is a major determinant of prognosis in pulmonary arterial hypertension.**


  - Over-secretion of ANP is correlated with hypoxia, which leads to **pulmonary vasoconstriction, pulmonary hypertension,** and right-heart overload [22, 23] .


  - It is also described in previous studies that approximately 14% of affected dogs also have evidence of associated **pulmonary arterial hypertension.**


  - Echocardiographic evaluation of the right **pulmonary artery distensibility index (RPAD index) was recently described as a valuable method for early detection and severity evaluation of pulmonary arterial hypertension** in dogs.


  - It is also described in previous studies that approximately 14% of affected dogs also have evidence of associated **pulmonary arterial hypertension.**


  - Echocardiographic evaluation of the right **pulmonary artery distensibility index (RPAD index) was recently described as a valuable method for early detection and severity evaluation of pulmonary arterial hypertension** in dogs.


  - Although endothelium-derived TXA 2 causes excessive **pulmonary vascular resistance by acting as a potent vasoconstrictor, mitogen of vascular smooth muscle cells and promoter of platelet aggregation, there is no current therapeutic strategy for pulmonary hypertension,** targeting TXA 2 formation.


  - These results suggest that TXA 2 synthase inhibitor may contribute to the improvement of **pulmonary hemodynamics in pulmonary hypertension by alleviating pulmonary** vasoconstriction and preventing thrombosis.


  - A new echocardiographic index, the right **pulmonary vein to pulmonary artery ratio (PV/PA) has been described for the detection of pulmonary venous hypertension.**


  - ARDS is defined as rapid onset of hypoxia with PaO 2 /fraction inspired oxygen (FiO 2 ) ratio <300 and bilateral **pulmonary infiltrates in the absence of left atrial hypertension.**


  - *Any underlying medical condition including chronic lung disease (asthma, chronic obstructive **pulmonary disease, interstitial lung disease, and bronchiectasis), chronic heart disease (coronary artery disease or congestive heart failure, hypertension, and valvulopathy), diabetes** mellitus, chronic kidney disease (with or without dialysis), neurologic disorders (epilepsy, cerebral palsy, dementia, or history of stroke), and chronic liver disease (hepatitis, cirrhosis, or hepatic failure).


  - The **pulmonary vein-to-pulmonary artery ratio (PV/PA) has been shown to be decreased in West Highland white terriers affected with pulmonary fibrosis in comparison with age-and breedmatched controls, suggesting its potential use in the non-invasive diagnosis of pulmonary hypertension** (PH).


  - Echocardiographic evaluations were performed on 127 healthy dogs (50 dogs had been included in a previous study of TAPSE), 55 dogs with heart disease but no **pulmonary hypertension (tricuspid regurgitant velocity <3 m/s) and 39 dogs with pulmonary hypertension** of various etiologies and severities (tricuspid regurgitant velocity >2.9 m/s).


  - Tricuspid annular plane excursion, fractional area change, tissue Doppler imaging and speckle-tracking echocardiography have been used to assess RV systolic function in healthy dogs and dogs with **pulmonary artery hypertension** (PAH).


  - Pulmonary-vein-to**-pulmonary-artery ratio (PV:PA) has been proposed as a method of identifying various stages of cardiovascular disease, including congestive heart failure (CHF) and pulmonary hypertension** (PH).


  - have pointed out that DPP-4 inhibitors that can alleviate **pulmonary artery remodeling and, finally, delay the development of pulmonary hypertension.**


  - neg., negative; pos., positive; COPD, chronic obstructive **pulmonary disease; AT, antitrypsin; PAH, pulmonary artery hypertension.**


  - ALI or **pulmonary embolism (PE) has been reported in humans and animals with intracranial disorders such as head trauma, brain tumor, intracranial hypertension** or cerebral compression.


  - The purpose of this study was to determine whether oral supplementation with L-arginine could reach plasma concentrations of L-citrulline, as a surrogate of nitric oxide generation, that are associated with **pulmonary vascular hemodynamic improvement in humans with pulmonary hypertension.**


  - Studies are underway to dose L-arginine in client-owned dogs with **pulmonary hypertension to see whether peak L-citrulline concentrations, or a specific % change in L-citrulline concentrations, are associated with clinical and hemodynamic improvement in pulmonary hypertension.**


  - One female child had end-stage **pulmonary hypertension, and it was unclear whether her respiratory distress was due to worsening pulmonary hypertension** or an infectious process.


  - ALI = acute lung injury; ARDS = acute respiratory distress syndrome; CMV = conventional mechanical ventilation; COPD = chronic obstructive **pulmonary disease; ECMO = extracorporeal membrane oxygenation; ET = endothelin; HFOV = high-frequency oscillatory ventilation; IAP = intraabdominal pressure; ICU = intensive care unit; IPAH = idiopathic pulmonary arterial hypertension; IPV = intrapulmonary percussive ventilation; PCP = pulmonary** capillary pressure; PEEP = positive end-expiratory pressure; SARS = severe acute respiratory syndrome; VAT = ventilator-associated tracheobronchitis.


  - Souza and coworkers analyzed **pulmonary artery pressure decay curves in patients with pulmonary hypertension due to either idiopathic pulmonary** arterial hypertension (IPAH; n = 12) or ARDS (n = 11) [24] .


  - So far, the mechanism underlying sildenafi l's eff ects on **pulmonary vascular remodeling and potassium channel activity in pulmonary artery smooth muscle cells (PASMCs) has not been clearly addressed in pulmonary hypertension secondary to increased pulmonary** blood fl ow.


  - Conclusions Oral sildenafi l attenuated **pulmonary vascular remodeling and upregulated Kv1.5 mRNA expression in the rats with pulmonary hypertension** secondary to left-to-right shunt.


  - • all persons aged ≥50 years, because in this group greatly increases the number of people belonging to high risk groups • adults and children who have chronic **pulmonary(including asthma), or cardiovascular (except isolated hypertension), renal, hepatic, neurological, hematologic or metabolic disorders (including diabetes** mellitus);


  - A compartment-specific transcriptional profiling approach revealed that lncRNA PAXIP1-AS1 regulates **pulmonary arterial hypertension** by modulating smooth muscle cell function (Jandl et al., 2019) .


  - Long non-coding RNAs influence the transcriptome in **pulmonary arterial hypertension:** the role of PAXIP1-AS1.


  - History of chronic obstructive **pulmonary disease (COPD) was non-significant higher but hypertension and cardiovascular** disease (CVD) was lower in men than in women.


  - The AT1R blocker valsartan has recently been reported to improve clinical outcomes for patients with chronic obstructive **pulmonary disease (COPD) complicated with pulmonary hypertension** (DOI:10.1183/13993003.congress-2019.PA2469).
</details>
    


<details>
  <summary> <strong> hypertension...hypertension </strong> </summary>
  
  - Specifically, NaCl consumption varied from roughly 5 g/d in Alaskan Eskimos where **hypertension was extremely rare, to 25 g/d in Northern Japanese, where nearly one third of the population exhibited hypertension.**


  - Specifically, NaCl consumption varied from roughly 5 g/d in Alaskan Eskimos where **hypertension was extremely rare, to 25 g/d in Northern Japanese, where nearly one third of the population exhibited hypertension.**


  - It is well accepted that excess salt exerts **hypertension and promotes renal damage in spontaneously hypertensive rats (SHR), which is used worldwide as a characterized experimental model of naturally occurring hypertension** [27] .


  - Another recent study indicated that prevalence of **hypertension among Arab American is 36.5% which is higher than the overall worldwide estimates of hypertension** prevalence [16] .


  - Another recent study indicated that prevalence of **hypertension among Arab American is 36.5% which is higher than the overall worldwide estimates of hypertension** prevalence [16] .


  - The effect of short rACE2 on acute **hypertension was examined in a model of angiotensin II-induced hypertension** as previously described by us [6, 7, 15] .


  - The effect of short rACE2 on acute **hypertension was examined in a model of angiotensin II-induced hypertension** as previously described by us [6, 7, 15] .


  - Experimental **hypertension studies have measured ACE2 in either the heart or kidney and/or plasma, and have reported that deletion or inhibition of ACE2 leads to hypertension,** whilst enhancing ACE2 protects against the development of hypertension, hence increasing ACE2 may be a therapeutic option for the management of high blood pressure in man.


  - Experimental **hypertension studies have measured ACE2 in either the heart or kidney and/or plasma, and have reported that deletion or inhibition of ACE2 leads to hypertension,** whilst enhancing ACE2 protects against the development of hypertension, hence increasing ACE2 may be a therapeutic option for the management of high blood pressure in man.


  - This review discusses the evidence regarding ACE2 in experimental **hypertension models and the association between circulating ACE2 activity and ACE2 polymorphisms with blood pressure and arterial hypertension** in man.


  - This review will discuss the evidence regarding ACE2 in experimental models of primary and secondary **arterial hypertension, the association between circulating ACE2 activity and blood pressure and hypertension** in man, as well as genetic association studies that have investigated ACE2 polymorphisms with blood pressure and/or arterial hypertension.


  - Although a potentially reversible cause of **hypertension can be identified in less than 10% of patients with hypertension,** the overall high prevalence of hypertension means that secondary forms can affect millions of patients worldwide.


  - Although a potentially reversible cause of **hypertension can be identified in less than 10% of patients with hypertension,** the overall high prevalence of hypertension means that secondary forms can affect millions of patients worldwide.


  - Dysregulation of ACE2 has been reported in experimental models of secondary **hypertension, such as endocrine hypertension** [deoxycorticosterone (DOCA)-salt, Ang II infusion], renal failure (subtotal nephrectomy), and renal artery stenosis (Goldblatt hypertension).


  - Dysregulation of ACE2 has been reported in experimental models of secondary **hypertension, such as endocrine hypertension** [deoxycorticosterone (DOCA)-salt, Ang II infusion], renal failure (subtotal nephrectomy), and renal artery stenosis (Goldblatt hypertension).


  - One study had a combined total of 2528 Chinese Han subjects with **hypertension and 2024 ethnicity matched normotensive controls and reported no association with hypertension** (Zhou and Yang, 2009 ).


  - In a small case-control study conducted in Anglo-Celtic Australian subjects with **hypertension (n = 152) or normotension (n = 193), four ACE2 SNPs were not associated with hypertension** (Benjafield et al., 2004) .


  - In a small case-control study conducted in Anglo-Celtic Australian subjects with **hypertension (n = 152) or normotension (n = 193), four ACE2 SNPs were not associated with hypertension** (Benjafield et al., 2004) .


  - Large carefully conducted clinical studies, designed with sufficient numbers and power to allow separate analyses of males and females are urgently needed in patients with **hypertension to more precisely clarify the potential role of ACE2 in blood pressure variation and hypertension,** and to uncover mechanisms that might offer novel treatment targets.


  - Large carefully conducted clinical studies, designed with sufficient numbers and power to allow separate analyses of males and females are urgently needed in patients with **hypertension to more precisely clarify the potential role of ACE2 in blood pressure variation and hypertension,** and to uncover mechanisms that might offer novel treatment targets.


  - Ang-(1-7) had no effect on the salt-induced hypertrophy in Ang II-independent **hypertension but it significantly reduced myocyte hypertrophy in Ang II-induced hypertension.**


  - Ang-(1-7) had no effect on the salt-induced hypertrophy in Ang II-independent **hypertension but it significantly reduced myocyte hypertrophy in Ang II-induced hypertension.**


  - Modulation of sACE2 activity in the nervous system (overexpression) also attenuated **hypertension [12] indicating a possibly protective role of sACE2 in hypertension.**


  - Modulation of sACE2 activity in the nervous system (overexpression) also attenuated **hypertension [12] indicating a possibly protective role of sACE2 in hypertension.**


  - We found that low eGFR, **hypertension, smoking status and history of atrial fibrillation were significantly associated with elevated TNFa levels, while elevated TNFR1 levels were associated with aging, low eGFR, hypertension,** diabetes and PVD, and elevated TNFR2 levels were associated with aging, low eGFR, diabetes and atrial fibrillation.


  - We found that low eGFR, **hypertension, smoking status and history of atrial fibrillation were significantly associated with elevated TNFa levels, while elevated TNFR1 levels were associated with aging, low eGFR, hypertension,** diabetes and PVD, and elevated TNFR2 levels were associated with aging, low eGFR, diabetes and atrial fibrillation.


  - For instance, **hypertension is the single most important modifiable risk factor for stroke, acting as a contributing factor in over 75 % of first time stroke patients [6] with hypertension** during acute stroke is associated with poorer clinical outcome [7] .


  - Our results indicate that the prevalence of **hypertension in the last 6 months before the survey has doubled from 3.6% in 2003 to 7.1% in 2008; and 17.5% of Chinese people had been diagnosed with hypertension** by medical doctors.


  - Our results indicate that the prevalence of **hypertension in the last 6 months before the survey has doubled from 3.6% in 2003 to 7.1% in 2008; and 17.5% of Chinese people had been diagnosed with hypertension** by medical doctors.


  - Although this was attributed to a possible "white-coat **hypertension" phenomenon, she was referred to the hypertension** clinic for further investigation and management of hypertension.


  - Although this was attributed to a possible "white-coat **hypertension" phenomenon, she was referred to the hypertension** clinic for further investigation and management of hypertension.


  - Liqourice-induced **hypertension with hypokalemia must be differentiated from other genetic deficiencies which may present with similar findings: three monogenetic types of mineralo-corticoid hypertension** have been identified, Liddle's syndrome, glucocorticoid-remediable hypertension, and apparent mineralo-corticoid excess, an autosomal recessive disorder with mutations in the 11β-HSD2 gene [36] [37] [38] [39] .


  - Liqourice-induced **hypertension with hypokalemia must be differentiated from other genetic deficiencies which may present with similar findings: three monogenetic types of mineralo-corticoid hypertension** have been identified, Liddle's syndrome, glucocorticoid-remediable hypertension, and apparent mineralo-corticoid excess, an autosomal recessive disorder with mutations in the 11β-HSD2 gene [36] [37] [38] [39] .


  - Zhao et al have found that renalase gene polymorphysim is associated with primary **hypertension, indicat-ing this gene may become the novel marker of genetic susceptibility in essential hypertension** [30] .


  - Zhao et al have found that renalase gene polymorphysim is associated with primary **hypertension, indicat-ing this gene may become the novel marker of genetic susceptibility in essential hypertension** [30] .


  - In turn, deficiencies in Ang-(1-7) that contribute to autonomic dysfunction were apparent in **hypertension (15) and aging (16) ; Ang-(1-7) deficiency in hypertension** was restored by ACE inhibitor treatment (17) or chronic Ang-(1-7) replacement (18) .


  - In turn, deficiencies in Ang-(1-7) that contribute to autonomic dysfunction were apparent in **hypertension (15) and aging (16) ; Ang-(1-7) deficiency in hypertension** was restored by ACE inhibitor treatment (17) or chronic Ang-(1-7) replacement (18) .


  - The endothelin system has been a target for therapeutic intervention due to its pathophysiological role in **hypertension, pulmonary arterial hypertension,** heart failure, renal disease, and diabetes.


  - Based on the dual-feedback system linking the endothelin system and the RAS, dual AT 1 R/ET A R antagonists have been developed and tested in the clinic for **pulmonary arterial hypertension, essential hypertension,** and chronic kidney disease (Murugesan et al., 2002 (Murugesan et al., , 2005 Neutel et al., 2008; Komers and Plotkin, 2016; Komers et al., 2017) ; however, it is still to be established whether the protective benefits outweigh the risk of adverse reactions.


  - Sacubitril/ valsartan has been evaluated in the management of **hypertension, heart failure with reduced ejection fraction (HFrEF), and heart failure with preserved ejection fraction (HFpEF) and has demonstrated clinical efficacy in the reduction of blood pressure in patients with essential hypertension** and without HFpEF and a reduction in hospitalizations and mortality for patients with HFrEF.


  - Sacubitril/ valsartan has been evaluated in the management of **hypertension, heart failure with reduced ejection fraction (HFrEF), and heart failure with preserved ejection fraction (HFpEF) and has demonstrated clinical efficacy in the reduction of blood pressure in patients with essential hypertension** and without HFpEF and a reduction in hospitalizations and mortality for patients with HFrEF.


  - Brain RAS hyperactivity has been implicated in the development and maintenance of **hypertension in several experimental and genetic animal models of hypertension,** such as SHRs, deoxycorticosterone acetate (DOCA)-salt rats, and transgenic mice overexpressing both angiotensinogen and renin human genes (Basso et al., 1981; Ganten et al., 1983; Davisson et al., 1998) .


  - Brain RAS hyperactivity has been implicated in the development and maintenance of **hypertension in several experimental and genetic animal models of hypertension,** such as SHRs, deoxycorticosterone acetate (DOCA)-salt rats, and transgenic mice overexpressing both angiotensinogen and renin human genes (Basso et al., 1981; Ganten et al., 1983; Davisson et al., 1998) .


  - Recombinant ACE2 and brain-selective APA inhibition are currently being tested clinically in **pulmonary arterial hypertension and overweight patients with hypertension,** respectively, based on the concept that ACE2 degrades Ang II and upregulates the protective Ang 1-7, whereas APA blockade prevents brain AT 1 R stimulation by Ang III.


  - 27 The most common cause of retinal detachment in a geriatric cat is **hypertension, with renal disease a common cause of secondary hypertension.**


  - 27 The most common cause of retinal detachment in a geriatric cat is **hypertension, with renal disease a common cause of secondary hypertension.**


  - Angiotensin-converting enzyme inhibitors (ACEi) are recommended to treat **hypertension associated with CKD to limit target-organ damage and especially glomerular hypertension.**


  - Angiotensin-converting enzyme inhibitors (ACEi) are recommended to treat **hypertension associated with CKD to limit target-organ damage and especially glomerular hypertension.**


  - Angiotensin-converting enzyme inhibitors (ACEi) are recommended to treat **hypertension associated with CKD to limit target-organ damage and especially glomerular hypertension.**


  - Angiotensin-converting enzyme inhibitors (ACEi) are recommended to treat **hypertension associated with CKD to limit target-organ damage and especially glomerular hypertension.**


  - Prevalence of **hypertension was 30% (9/30) and 4/30 had severe hypertension** (≥180 mm Hg).


  - Prevalence of **hypertension was 30% (9/30) and 4/30 had severe hypertension** (≥180 mm Hg).


  - Prevalence of **hypertension was different in diabetic cats without concurrent diseases (0%), diabetic cats with a disease not associated with hypertension** (29%) and those with a disease known to increase SBP (56%)( P = 0.023).


  - Prevalence of **hypertension was different in diabetic cats without concurrent diseases (0%), diabetic cats with a disease not associated with hypertension** (29%) and those with a disease known to increase SBP (56%)( P = 0.023).


  - Aims of this study were to identify the frequency of **hypertension in dogs with DM, and characterize the relationship between hypertension** and age, sex, reproductive status, duration of DM, body condition score, concurrent diseases and treatment with corticoids.


  - Aims of this study were to identify the frequency of **hypertension in dogs with DM, and characterize the relationship between hypertension** and age, sex, reproductive status, duration of DM, body condition score, concurrent diseases and treatment with corticoids.


  - Systemic **arterial hypertension is not frequently recognized in dogs with mitral valve degeneration (MVD), although borderline hypertension** is difficult to assess, mainly because of different measurement techniques, inter-operator variability and, most importantly, examination-related stress.


  - Chen et al [2] and Huang et al [4] also reported that 32-51% of the hospitalized patients with confirmed COVID-19 had **hypertension, diabetes, and cardiovascular diseases, while 10 of the 28 Korean patients (35.7%) had at least 1 underlying disease, including hypertension** and diabetes.


  - Since the most common cause of cardiomyocyte hypertrophy is **hypertension, and since an increased Ang II level in hypertension** is a critical pro-hypertrophy trigger, we next explored whether there might be an association between HGF and Ang II in cardiomyocytes.


  - Since the most common cause of cardiomyocyte hypertrophy is **hypertension, and since an increased Ang II level in hypertension** is a critical pro-hypertrophy trigger, we next explored whether there might be an association between HGF and Ang II in cardiomyocytes.


  - Early studies in our laboratory demonstrated that acute PE of hemorrhagic and fulminant type occurred accompanying severe **hypertension and bradycardia (Cushing responses) in rats following cerebral compression (CC) or intracranial hypertension** (ICH).


  - Early studies in our laboratory demonstrated that acute PE of hemorrhagic and fulminant type occurred accompanying severe **hypertension and bradycardia (Cushing responses) in rats following cerebral compression (CC) or intracranial hypertension** (ICH).


  - Seven of our 28 patients had associated comorbid diseases; these include two cases of malignancy including one associated with **hypertension, and one case each of hypertension,** chronic lung disease, valvular heart disease, thalassemia A close-up view of a chest radiograph in the posteroanterior projection obtained 4 days after fever onset shows a subtle ground glass density in the lateral aspect of the right upper lung (arrows) (radiographic score = 1).


  - Seven of our 28 patients had associated comorbid diseases; these include two cases of malignancy including one associated with **hypertension, and one case each of hypertension,** chronic lung disease, valvular heart disease, thalassemia A close-up view of a chest radiograph in the posteroanterior projection obtained 4 days after fever onset shows a subtle ground glass density in the lateral aspect of the right upper lung (arrows) (radiographic score = 1).


  - More than half of patients had history of **hypertension, which was from 59% to 63% and the frequency of hypertension** in ischemic stroke was not much different in three time period.


  - More than half of patients had history of **hypertension, which was from 59% to 63% and the frequency of hypertension** in ischemic stroke was not much different in three time period.


  - Underlying disease status among the 32 patients were no DM, **hypertension, or COD (n = 11); DM without hypertension** or COD (n = 5); DM and hypertension without COD (n = 5); and DM or hypertension with COD (n = 11).


  - Underlying disease status among the 32 patients were no DM, **hypertension, or COD (n = 11); DM without hypertension** or COD (n = 5); DM and hypertension without COD (n = 5); and DM or hypertension with COD (n = 11).


  - Of the 11 case-patients with no DM or **hypertension, 100% survived, as did all patients with DM but without hypertension** or COD.


  - Of the 11 case-patients with no DM or **hypertension, 100% survived, as did all patients with DM but without hypertension** or COD.


  - In comparison, 2 (40%) of 5 with DM and **hypertension but without COD died, and 9 (82%) of 11 with DM or hypertension** and with COD died (Appendix Table) .


  - In comparison, 2 (40%) of 5 with DM and **hypertension but without COD died, and 9 (82%) of 11 with DM or hypertension** and with COD died (Appendix Table) .


  - High occurrence of **hypertension was detected in the AMI patients (total patients with hypertension:** n=8, 15.09%; AMI group: n=4, 66.67%; cardiac marker abnormalities group: n=2, 8.33%; without cardiac marker abnormalities group: n=2, 8.69%; p=0.001).


  - High occurrence of **hypertension was detected in the AMI patients (total patients with hypertension:** n=8, 15.09%; AMI group: n=4, 66.67%; cardiac marker abnormalities group: n=2, 8.33%; without cardiac marker abnormalities group: n=2, 8.69%; p=0.001).
</details>
    


<details>
  <summary> <strong> comorbidities...hypertension </strong> </summary>
  
  - Third, although aging impacts immunity to viral infection [30] , patients older than 60 years of age usually have some **comorbidities, such as hypertension, diabetes,** or chronic bronchitis, and seek medical attention only when these primary diseases worsen when combined with ARI.


  - The overseas KT recipients also suffered from more **comorbidities: they had a higher CCI score (1.01 vs 0.73, p<0.001), as well as higher percentages of diabetes (14.2% vs 8.0%, p<0.001) and hypertension** (38.6% vs 26.5%, p<0.001).


  - The enrolled patients had various **comorbidities; three patients were complicated with anxiety and depression requiring treatment, sixteen had hypertension, five had diabetes,** seven had peptic ulcers, and one had Sjogren's syndrome.


  - Patients with **comorbiditie**s (hypertension,** diabe**tes, chronic lung diseases, cardiovascular disorders or neuromuscular diseases) were at greater risk of hospitalization after influenza viral infection.


  - Except asthmatic subgroup, **comorbidities such as hypertension, diabetes** mellitus, hyperlipidemia and coronary artery disease were frequently observed.


  - Of the 281 confirmed cases, 160 (56.9%) involved **comorbidities, wherein diabetes mellitus, hypertension, ischemic heart disease, congestive** heart failure, end-stage renal disease and chronic kidney disease were significantly associated with mortality from MERS-CoV and two or three comorbidities significantly affected the fatality rates from MERS-CoV.


  - Pooled estimates of the OR were greater than the value of 1 for all five **comorbidities, including diabetes mellitus (n = 8 studies), renal disease (6 studies), respiratory disease (5 studies), heart disease (5 studies) and hypertension** (5 studies).


  - A total of 26 cases with fatalities were used to disclose the dangerous **comorbidities, showing that the major comorbidities are hypertension (53.8%), diabetes** (42.3%), coronary heart disease (19.2%), cerebral infarction (15.4%), chronic bronchitis (19.2%) and Parkinson's disease (7.7%) (Figure 2 ).


  - From the analysis of the fatal cases of this novel coronavirus pneumonia, the **comorbidities of hypertension, diabetes,** coronary heart disease, cerebral infarction, and chronic bronchitis were found to be dangerous factors that resulted in death.


  - About the **comorbidities, patients with AMI and sepsis have significant higher rates of hypertension, dyslipidemia,** diabetes, peripheral vascular disease, congestive heart failure, end-stage renal disease, cerebral vascular accident and chronic obstructive pulmonary disease (all P < 0.05 Conclusions In patients with ACS, clinical manifestations of AHF at admission constitute a strong predictor of adverse outcome in the follow up that may be significantly modified by the coronary revascularization strategy


  - Patient medical records were reviewed and clinical data were extracted, including demographic characteristics, body mass index (BMI), smoking history, diagnostic method, Sequential Organ Failure Assessment (SOFA) score, nosocomial infection, and **comorbiditie**s (hypertension,** diabe**tes mellitus, chronic obstructive pulmonary disease [COPD], chronic kidney disease, chronic liver disease, current or previous solid organ malignancy).


  - 26 We have encountered poor prognosis and long hospital stay in patients with scrub typhus if AKI was accompanied, especially in patients with **comorbidities such as diabetes mellitus (DM), hypertension** and CKD.


  - The most common **comorbidities reported were hypertension** (41%), metabolic disorders (33%), and chronic lung disease (30%).


  - As has been shown by others, **comorbidities such as chronic lung disease, diabetes, and hypertension** likely contributed to the poor survival of patients with influenza in our population.


  - The second patient, a 39-year-old at 24 weeks gestation, had several **comorbidities, including end stage renal disease, hypertension,** and hemodialysis.


  - While most of the patients in this study had some kind of underlying **comorbidities (52.7% have at least one comorbidities), around 38% of all patients had more than one comorbidities with the most common being obesity, diabetes and hypertension** (which occurred in more than 50% of those with any underlying comorbidity) ( Table 1) .


  - Severe and fatal disease is strongly linked to underlying **comorbidities including diabetes mellitus, hypertension, obesity,** and chronic cardiac, pulmonary, or renal disease [23] .


  - The CFR was also increased in those with **comorbidities including cardiovascular, diabetes, chronic respiratory disease, hypertension,** and cancer.


  - A total of 75% (n = 24) of affected patients had **comorbidities that included diabetes mellitus, hypertension, and cardiovascular,** neurological and immunodeficiency diseases.


  - The most common **comorbidities were hypertension in 54(32%) patients, cardiac failure in 36(21.3%) situations and diabetes** in 32(18.9%) cases.


  - Pneumonia is a common illness that is life-threatening to the elderly, especially those with **comorbidities such as diabetes or hypertension** [4] .


  - Potential predictors examined in bivariate analysis included age, sex, race/ethnicity, length of hospital stay, time from symptom onset to hospital admission, ICU admission, presence of **comorbidities (≥ 1, ≥ 2 and ≥ 3 comorbidities), individual comorbiditie**s (hypertension,** cardiac disease, lung disease, obes**ity, immunodeficiency and neurologic disorder), and the presence of individual viral pathogens (influenza viruses, parainfluenza viruses, coronaviruses, HMPV, RSV and rhinoviruses).


  - ICD-9-CM codes were also used to identify patients treated with IABP who underwent PCI or CABG, and to identify patients treated with IABP who had baseline **comorbidities of hypertension (HTN) (ICD-9-CM: 401-405), diabetes** mellitus (DM) (ICD-9-CM: 250), hyperlipidemia (ICD-9-CM: 272), myocardial infarction (MI) (ICD-9-CM: 410), and cerebrovascular attack (CVA) (ICD-9-CM: 430-438).


  - The objective of this study is to systematically review the existing literature on the prevalence of the most common non-communicable **comorbidities related to the cluster of metabolic syndromes-associated diseases, such as diabetes mellitus, heart diseases, hypertension, asthma, stroke** and obesity in flavivirus infections and to evaluate the difference of their prevalence in severe vs. non-severe clinical outcomes to infection.


  - Data extracted from the selected studies in duplicate by two reviewers and included the first author's name, publication date, country, dates of recruitment, total sample size (divided to males and females), age estimates (from reported mean, median or the mid-point for age range of the highest subject frequency), procedures for case identification, type of flavivirus infection, severity of infection, prevalence of clinical manifestations (mild symptoms such as fever, headache, muscle pain, rash, and malaise together with severe symptoms as described below) and percentage of **comorbidities including diabetes (both type I and type II, if mentioned), hypertension,** heart diseases (due to the small sample size of individual conditions, we (Table 1) .


  - Results of Egger's test, however, for most of the associations between DENV or WNV and the **comorbidities showed p>0.1 (except for the prevalence of hypertension** in DENV and heart diseases in WNV, p = 0.03), given the assumption for evidence of smallstudy effects is based on p<0.1 as previously reported [109] .


  - However, in WNV studies, the among-studies prevalence of **comorbidities varied by 3-to 12-fold for diabetes, hypertension, heart diseases and stroke** with low to moderate I 2 indexes.


  - Similarly, most of the pilgrims are older people (>50 years) with **comorbidities such as hypertension, diabetes,** and liver diseases [5, 6] .


  - The concomitant **comorbidities at the start of continuous ambulatory peritoneal dialysis (CAPD) encompassed diabetes mellitus (n = 179, 34.6%), hypertension (n = 419, 85.2%), and cardiovascular** disease (n = 186, 37.9%).


  - The **comorbidities included diabetes mellitus (DM), hypertension,** CVD and Charlson Comorbidity Index, which is one of the most commonly used scoring systems to predict the risk of mortality.


  - The concomitant **comorbidities at the start of CAPD encompassed diabetes mellitus (n = 179, 34.6%), hypertension (n = 419, 85.2%), and cardiovascular** disease (n = 186, 37.9%).


  - The most frequent **comorbidities recorded were hypertension (43.6%), psychiatric disorders including anxiety and/or depression (42.0%), heart failure (32.8%), ischemic heart disease (28.3%), and diabetes** mellitus (28.0%).


  - We adjusted for **comorbidities, such as hypertension, diabetes,** stroke, and arthritis, in order to overcome this unintended effect.


  - 10 Many infected individuals exhibit **comorbidities (e.g., diabetes, hypertension, asthma, and obesity)**, rendering them more susceptible to complications.


  - Data extracted from the selected studies included the first author's name, publication date, country, dates of recruitment, ethnicity, total sample size (divided to males and females), age, prevalence of symptoms (including fever, shortness of breath, sore throat, cough and Acute Physiology and Chronic Health Evaluation II, APACHEII) and percentage of **comorbidities at the time of presentation (including obesity, immunosuppression or HIV and heart conditions such as hypertension, cardiovascular** diseases and/or coronary arterial diseases) (Tables 1 and 2).


  - A recent study noted that, compared to subjects with no **comorbidities, severe cases of influenza are more likely to manifest with obesity (OR for mortality 2.74, 95%CI: 1.56-4.80), cardiovascular diseases (2.92, 1.76-4.86), hypertension** (1.49, 1.10-2.01) and neuromuscular disease (2.68, 1.91-3.75).


  - Personal data included sex, age, epidemiological history, history of smoking, and **comorbidities [e.g., chronic obstructive pulmonary disease (COPD), cancer, hypertension and/or diabetes]**.


  - Twenty patients (25.6%) had **comorbidities, of which hypertension** was the most common.


  - The independent variables included the following major diseases and/or **comorbidities: BBP, cancer, liver cirrhosis, HIV/AIDS, acute myocardial infarction, systemic arterial hypertension, dilated cardiomyopathy, pulmonary** embolism, chronic obstructive pulmonary disease, diabetes mellitus, sepsis and septic shock, chronic kidney failure, and tuberculosis.


  - The most frequent **comorbidities among the patients were diabetes (58.33%), hypertension** (37.50%), and chronic kidney disease (16.66%) ( Table 2 ).


  - -Particular attention should also be paid to certify that staff deployed to provide psychological-psychiatric care in institutions are informed and/or sensitized of known potential risk factors for a more severe course of a Covid-19 infection of their own (advanced age, somatic **comorbidities, chronic respiratory diseases, hypertension,** cancer, known immune deficiencies etc.).


  - They showed no infi ltrations on chest X-ray, had moderately to very severely impaired lung function according to GOLD stages II-IV and reported various **comorbidities (arterial hypertension, cor pulmonale, obstructive sleep apnea, cardiomyopathy and ischemic stroke)**.


  - Interestingly, other statistically signifi cant associated **comorbidities were cancer, CHF, arrhythmias, and hypertension,** which were more commonly seen in the HDLVEF group.


  - Meanwhile, numbers of patients with COVID-19 infection had chronic **comorbidities, mainly hypertension, diabetes** and cardiovascular disease, which is similar to MERS-COV population.


  - All the six AMI patients were aged >60 years; five of them had two or more underlying **comorbiditie**s (hypertension,** diabe**tes, cardiovascular diseases, and chronic obstructive pulmonary disease).


  - The **comorbidities with high incidence in the elderly like diabetes mellitus (DM), hypertension, coronary** artery disease (CAD), chronic obstructive pulmonary disease (COPD) and malignant tumor were all lower than 5.0%.


  - Patients who are older (≥60 years) or carry **comorbidities of disorders (69.57% vs 11.29%, p<0.001), especially hypertension** (39.13% vs 12.90%, p=0.0007) or Coronary heart disease (21.74% vs 4.84%, p=0.018) are more easily developed ARF.


  - Severe infiltration of lymphocytes in tubulointerstitium was only observed in one patient who carries **comorbidities of hypertension (>10 years) and coronary** disease (>10 year).


  - While minimal infiltration was seen in three cases who are **comorbidities of disorders like hypertension** and gout, and the other two cases are absence tubulointerstitial lymphocyte infiltration, moreover, no obvious lymphocytes were seen in glomeruli, suggesting lymphocyte infiltration is uncommon after viral infection.


  - Consistent with recent reports, our results suggest that 2019-nCoV is more likely to infect elderly people with chronic **comorbidities as a result of the weaker immune functions of these patients [5, 6] , and 2019-nCoV-associated death is also related to elder age and underlying illnesses, especially hypertension, diabetes,** and cardiac disease.


  - Thirty-nine cases (38.2%) had **comorbidities, including hypertension, diabetes,** coronary artery disease, tuberculosis, hepatitis B and chronic bronchitis.


  - Age is described by intervals of decades (0-9; 10-19; up to ≥ 80 years old) and the **comorbidities are: hypertension, diabetes,** cardiovascular disease, chronic respiratory disease and cancer.
</details>
    

...


<details>
  <summary> <strong> hypertension...portal </strong> </summary>
  
  - However, symptoms associated with portal **hypertension must be monitored in animals with primary portal** vein hypoplasia.


  - However, symptoms associated with portal **hypertension must be monitored in animals with primary portal** vein hypoplasia.
</details>
    


<details>
  <summary> <strong> artery...hypertension </strong> </summary>
  
  - He reasoned in his thesis that the severity of the brain pathology in these patients correlates with the diameter of their aneurismal internal carotid **artery, which is similar to the situation of a cirrhotic liver and the dilated portal vein due to portal hypertension.**


  - Doppler evaluation of the renal **artery has been performed in humans to identify renal artery stenosis or kinking of an artery as potential causes of renal hypertension.**
</details>
    


<details>
  <summary> <strong> mellitus...hypertension </strong> </summary>
  
  - Diabetes **mellitus (DM), hypertension** (HTN), cardiac diseases, renal disease and bronchial asthma were the most frequent comorbid disorders (Table 4) .
</details>
    


<details>
  <summary> <strong> hypertension...artery </strong> </summary>
  
  - Paroxysmal **hypertension is a known complication following head and neck radiation, and can be attributed to damage to the carotid artery** baroreceptors [13] .
</details>
    


<details>
  <summary> <strong> hyperlipidemia...hypertension </strong> </summary>
  
  - Increased ALP (86.7%) and ALT (40%), **hyperlipidemia (45.6%), low urinary density (57.5%), thrombocytosis (30.2%) and hypertension** (51.4%) were reported.
</details>
    

In [152]:
for paper in sorted(covid_papers, key=lambda x: -len(x['snippets'])):
    if not paper['snippets']: break
    dm(f"### {paper['metadata.title'][0]['text']}")
    for snippet in paper['snippets']:
       dm(f'- {snippet}')
    dm('---')
    

### High-Resolution Analysis of Coronavirus Gene Expression by RNA Sequencing and Ribosome Profiling

- {'sent': '**Ribosome profiling is a technique which** exploits the capacity of the translating ribosome to protect around 30 nucleotides of mRNA from ribonuclease digestion.', 'lgd': {'word': 'Ribosome', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'which'}}

- {'sent': '**Ribosome-protected mRNA fragments are purified, subjected to deep sequencing and mapped back to the transcriptome to give a global "snap-**shot" of translation.', 'lgd': {'word': 'Ribosome', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'snap'}}

- {'sent': '**Here we apply ribosome profiling to cells infected with Murine coronavirus, mouse hepatitis virus, strain A59 (MHV-A59), a model coronavirus** in the same genus as SARS-CoV and MERS-CoV.', 'lgd': {'word': 'Here', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'coronavirus'}}

- {'sent': '**Ribosome profiling is emerging as a powerful technique** to monitor translation in living cells at sub-codon resolution.', 'lgd': {'word': 'Ribosome', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'technique'}}

...

- {'sent': '**The focus so far has only been on HIV-1, RSV,** and HCV [260, [262] [263] [264] [265] .', 'lgd': {'word': 'The', 'seed': '1'}, 'rgd': {'seed': '1', 'word': 'RSV'}}

- {'sent': '**Stapled peptides (1) can** inhibit PPIs; (2) are more specific for their targets than small-molecule drugs, which also decreases the risk of unwanted side-effects; (3) can target diseases that are otherwise difficult to treat, referred to as "undruggable"; (4) can be modified easily to enhance membrane permeability, potency, and half-life; (5) have a short market time [253, 266, 267] .', 'lgd': {'word': 'Stapled', 'seed': '1'}, 'rgd': {'seed': '1', 'word': 'can'}}

- {'sent': '**Autophagy is a cellular process** that recycles excess or damaged cellular material to maintain the energy levels of the cell and ensure its survival.', 'lgd': {'word': 'Autophagy', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'process'}}

- {'sent': '**Left: The binding of biotin to avidin occurs in a deep groove,** while the interaction between the human growth hormone (hGH) and the hGH receptor (hGHR) occurs over a larger, flatter area [254] a defence mechanism to promote viral propagation [274] [275] [276] .', 'lgd': {'word': 'Left', 'seed': 'a'}, 'rgd': {'seed': 'a', 'word': 'groove'}}

KeyboardInterrupt: 

In [68]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True